# The MacGyver MUD: A Journey Into Active Inference

## Interactive Deep Dive: From Uncertainty to Geometric Diagnostics

**Version**: 2.0 "Silver Rails"

**Estimated Time**: 2-3 hours

---

### What You'll Discover

You're locked in a room. There's a door (might be locked) and a window (escape route, but costly).

**The Question**: How should an intelligent agent decide what to do?

By the end of this notebook, you'll understand:

1. 🤖 **Active Inference** - How agents balance exploration and exploitation
2. 📐 **Pythagorean Means** - 2500-year-old math applied to modern AI
3. 🎯 **The k≈0 Revelation** - Why ALL simple skills are "specialists"
4. 🌈 **Multi-Objective Evolution** - Filling geometric gaps with balanced skills
5. 🔬 **Diagnostic-Driven Design** - A meta-pattern for innovation

### How This Notebook Works

- ✅ **Interactive**: Sliders, calculators, live queries
- ✅ **Real Data**: Everything uses actual Neo4j database
- ✅ **Progressive**: Concrete examples → Mathematical formulas
- ✅ **Visual**: Graphs, plots, animations throughout
- ✅ **Tested**: Checkpoints to verify understanding

---

*"Measure what is measurable, and make measurable what is not so." — Galileo*

*We've made decision strategies measurable through geometry.*

Let's begin...

---

# PART 0: Setup & Orientation

**Time**: 5 minutes

**Goals**:
- Connect to Neo4j database
- Import required libraries
- Configure learning preferences
- Understand what we'll discover

## 0.1 Library Imports & Configuration

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import beta
import warnings
warnings.filterwarnings('ignore')

# Graph database
from neo4j import GraphDatabase

# Interactive widgets
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML, Image, clear_output

# Graph visualization
import networkx as nx

# 3D visualization (optional)
try:
    import plotly.graph_objects as go
    import plotly.express as px
    PLOTLY_AVAILABLE = True
except ImportError:
    PLOTLY_AVAILABLE = False
    print("Note: Plotly not available. 3D visualizations will be skipped.")

# Plotting configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Color scheme
COLORS = {
    'sense': '#3498db',      # Blue - sensing skills
    'act': '#e74c3c',        # Red - action skills
    'balanced': '#2ecc71',   # Green - balanced skills
    'specialist_zone': '#ffcccc',
    'balanced_zone': '#ccffcc'
}

print("✓ Libraries loaded successfully")
print(f"✓ Numpy version: {np.__version__}")
print(f"✓ Pandas version: {pd.__version__}")
print(f"✓ Matplotlib available: {plt.matplotlib.__version__}")
print(f"✓ Plotly available: {PLOTLY_AVAILABLE}")

## 0.2 Neo4j Database Connection

In [ ]:
# Neo4j connection configuration
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "password"

# Connect to Neo4j
try:
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    
    # Test connection and get database stats
    with driver.session() as session:
        result = session.run("""
            MATCH (n)
            RETURN count(n) as node_count
        """)
        node_count = result.single()['node_count']
        
        # Get node type breakdown
        result = session.run("""
            MATCH (n)
            RETURN labels(n)[0] as label, count(*) as count
            ORDER BY count DESC
        """)
        
        print("✓ Connected to Neo4j successfully!")
        print(f"  Total nodes in database: {node_count}")
        print("\n  Node breakdown:")
        for record in result:
            print(f"    {record['label']}: {record['count']}")
    
    NEO4J_CONNECTED = True
    
except Exception as e:
    print(f"✗ Failed to connect to Neo4j: {e}")
    print("  Make sure Neo4j is running: make neo4j-start")
    print("  And database is initialized: make init")
    NEO4J_CONNECTED = False
    driver = None

## 0.3 Helper Functions

In [ ]:
def run_query(query, **params):
    """Execute a Neo4j query and return results as list of dicts"""
    if not NEO4J_CONNECTED:
        print("⚠ Neo4j not connected. Cannot run query.")
        return []
    
    with driver.session() as session:
        result = session.run(query, **params)
        return [dict(record) for record in result]

def style_plot(ax, title, xlabel, ylabel):
    """Apply consistent styling to matplotlib plots"""
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xlabel(xlabel, fontsize=12)
    ax.set_ylabel(ylabel, fontsize=12)
    ax.grid(alpha=0.3)

def silver_k_explore(goal_value, info_gain):
    """Calculate k_explore coefficient using Pythagorean means"""
    epsilon = 1e-10
    gm = np.sqrt(goal_value * info_gain + epsilon)
    am = (goal_value + info_gain) / 2.0 + epsilon
    return gm / am

def silver_k_efficiency(benefit, cost):
    """Calculate k_efficiency coefficient"""
    epsilon = 1e-10
    cost_inv = 1.0 / (cost + epsilon)
    gm = np.sqrt(benefit * cost_inv + epsilon)
    am = (benefit + cost_inv) / 2.0 + epsilon
    return gm / am

def score_skill(skill, belief_locked):
    """Calculate Expected Free Energy for a skill"""
    cost = skill['cost']
    belief_unlocked = 1 - belief_locked
    expected_goal = skill.get('goal_info', skill.get('goal_fraction', 0)) * belief_unlocked
    expected_info = skill.get('info_gain', skill.get('info_fraction', 0))
    efe = cost - expected_goal - expected_info
    return efe

print("✓ Helper functions defined")

## 0.4 Learning Path Selector

In [ ]:
display(Markdown("""
### Choose Your Learning Depth

This notebook adapts to your background. Select your comfort level:

- **Intuitive**: Minimal math, focus on concepts and visualizations
- **Computational**: Basic formulas and code walkthrough
- **Mathematical**: Full derivations and proofs

Don't worry - you can always expand optional sections later!
"""))

learning_depth = widgets.RadioButtons(
    options=['Intuitive (minimal math)', 
             'Computational (basic formulas)', 
             'Mathematical (full derivations)'],
    value='Computational (basic formulas)',
    description='Your path:',
    style={'description_width': 'initial'}
)

display(learning_depth)

# This will be used to show/hide detailed mathematical sections
def get_depth_level():
    if 'Intuitive' in learning_depth.value:
        return 'intuitive'
    elif 'Computational' in learning_depth.value:
        return 'computational'
    else:
        return 'mathematical'

---

# PART 1: The MacGyver Problem

**Time**: 10 minutes

**Goals**:
- Understand the locked room scenario
- Build intuition about uncertainty
- Explore available skills
- Realize why beliefs matter

**No math yet** - just concrete problem-solving!

## 1.1 The Scenario

In [ ]:
display(Markdown("""
## 🚪 The Locked Room Scenario

Imagine: You wake up in a room with:

### Two Possible Exits:

🚪 **A Door**
- Might be locked (you don't know yet!)
- If unlocked: Quick and easy escape
- If locked: Wasted effort trying

🪟 **A Window**
- Always accessible (guaranteed exit)
- But you're on 2nd floor (risky, costly)
- Should be last resort

### Your Objective:
**Escape as quickly and safely as possible**

### The Challenge:
**You're UNCERTAIN about the door state!**
- Is it locked? 🔒
- Is it unlocked? 🔓
- How can you decide what to do?

This uncertainty makes the problem interesting...
"""))

## 1.2 Interactive Room Graph from Neo4j

In [ ]:
def visualize_room_graph():
    """Query and visualize the room structure from Neo4j"""
    if not NEO4J_CONNECTED:
        print("⚠ Neo4j not connected. Showing conceptual diagram instead.")
        # Show simple diagram
        G = nx.DiGraph()
        G.add_edge('stuck_in_room', 'escaped_via_door', label='door unlocked')
        G.add_edge('stuck_in_room', 'escaped_via_window', label='use window')
        G.add_edge('stuck_in_room', 'still_stuck', label='door locked')
    else:
        # Query actual graph
        results = run_query("""
            MATCH (start:State {name: 'stuck_in_room'})
            OPTIONAL MATCH (start)-[r]->(other)
            RETURN start.name as start_name, type(r) as rel_type, other.name as end_name
        """)
        
        G = nx.DiGraph()
        for record in results:
            if record['end_name']:
                G.add_edge(record['start_name'], record['end_name'], 
                          label=record['rel_type'])
    
    # Visualize
    plt.figure(figsize=(14, 8))
    pos = nx.spring_layout(G, k=2, iterations=50)
    
    # Color nodes
    node_colors = []
    for node in G.nodes():
        if 'escaped' in node:
            node_colors.append('#2ecc71')  # Green for success
        elif 'stuck' in node:
            node_colors.append('#e74c3c')  # Red for stuck
        else:
            node_colors.append('#3498db')  # Blue for neutral
    
    nx.draw(G, pos, with_labels=True, node_color=node_colors,
            node_size=4000, font_size=11, font_weight='bold',
            arrows=True, edge_color='gray', width=2.5,
            arrowsize=20, alpha=0.9)
    
    # Edge labels
    edge_labels = nx.get_edge_attributes(G, 'label')
    nx.draw_networkx_edge_labels(G, pos, edge_labels, font_size=10)
    
    plt.title("Room State Graph" + (" (from Neo4j Database)" if NEO4J_CONNECTED else " (Conceptual)"),
              fontsize=16, fontweight='bold', pad=20)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    print("\n💡 Key Insight: The graph shows possible states and transitions.")
    print("   Notice: There are multiple paths to escape!")

visualize_room_graph()

## 1.3 Interactive Quiz: What Would YOU Do?

In [ ]:
display(Markdown("""
### 🤔 Decision Time!

You're in the room. You don't know if the door is locked or not.

**What would you do first?**
"""))

quiz_choice = widgets.RadioButtons(
    options=['A) Immediately try to open the door',
             'B) First peek through the keyhole to check if locked',
             'C) Immediately go to the window and escape'],
    description='Your choice:',
    style={'description_width': 'initial'}
)

submit_button = widgets.Button(
    description="Submit Answer",
    button_style='primary',
    icon='check'
)

output = widgets.Output()

def check_answer(b):
    with output:
        clear_output()
        if 'peek through the keyhole' in quiz_choice.value:
            print("✓ Smart thinking! Gathering information before acting is often wise.")
            print("\n💡 This is called 'exploration' - reducing uncertainty.")
            print("   By peeking first, you learn the door's state before committing effort.")
            print("\n   If door is unlocked → try it (cheap success!)")
            print("   If door is locked → skip to window (avoid wasted effort)")
        elif 'try to open' in quiz_choice.value:
            print("⚠ Bold move! But think about it...")
            print("\n   What if the door is locked? You've wasted effort trying.")
            print("   Gathering information first (peeking) might save effort.")
            print("\n💡 Though... if you're CONFIDENT the door is unlocked, trying immediately makes sense!")
            print("   So your BELIEF about the door state matters!")
        else:
            print("⚠ Very cautious! But perhaps TOO cautious?")
            print("\n   The window is risky and costly (2nd floor).")
            print("   What if the door was unlocked all along? You'd take unnecessary risk.")
            print("\n💡 Checking the door first (peek or try) could save you from danger!")
        
        print("\n" + "="*70)
        print("🎯 KEY INSIGHT: The 'best' action depends on your BELIEFS about the world!")
        print("="*70)

submit_button.on_click(check_answer)
display(quiz_choice, submit_button, output)

## 1.4 Uncertainty Matters: Interactive Belief Explorer

In [ ]:
display(Markdown("""
### 🎲 How Beliefs Change Decisions

Let's explore how your **belief** about the door state affects the best choice.

**Adjust the slider**: What do you believe about the door?
"""))

belief_slider = widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1,
    step=0.05,
    description='Belief door is LOCKED:',
    style={'description_width': 'initial'},
    continuous_update=True,
    readout_format='.0%'
)

output_recommendation = widgets.Output()

def show_recommendation(change):
    belief = belief_slider.value
    with output_recommendation:
        clear_output(wait=True)
        
        # Visual belief distribution
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        
        # Left: Belief bar chart
        states = ['Unlocked', 'Locked']
        probabilities = [1 - belief, belief]
        colors = ['#2ecc71', '#e74c3c']
        
        bars = ax1.bar(states, probabilities, color=colors, alpha=0.7, 
                       edgecolor='black', linewidth=2)
        ax1.set_ylabel('Probability', fontsize=12)
        ax1.set_title('Your Belief Distribution', fontsize=14, fontweight='bold')
        ax1.set_ylim(0, 1)
        ax1.grid(axis='y', alpha=0.3)
        
        # Add probability labels
        for bar, prob in zip(bars, probabilities):
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height,
                    f'{prob:.0%}', ha='center', va='bottom', 
                    fontsize=14, fontweight='bold')
        
        # Right: Recommendation
        ax2.axis('off')
        
        if belief < 0.3:
            recommendation = "🚪 TRY THE DOOR"
            reason = "Door is probably UNLOCKED\nQuick escape likely!"
            color = '#2ecc71'
        elif belief > 0.7:
            recommendation = "👁 PEEK FIRST"
            reason = "Door is probably LOCKED\nDon't waste effort trying.\nCheck first, then decide."
            color = '#3498db'
        else:
            recommendation = "🤔 UNCERTAIN"
            reason = "You don't know!\nGathering info (peek)\ncould help decision."
            color = '#f39c12'
        
        ax2.text(0.5, 0.6, recommendation, 
                ha='center', va='center', fontsize=28, fontweight='bold',
                color=color, bbox=dict(boxstyle='round', facecolor=color, alpha=0.2))
        ax2.text(0.5, 0.3, reason,
                ha='center', va='center', fontsize=14, style='italic')
        
        plt.tight_layout()
        plt.show()
        
        # Text summary
        print("\n" + "="*70)
        print(f"Belief: {belief:.0%} LOCKED, {1-belief:.0%} UNLOCKED")
        print(f"Recommendation: {recommendation}")
        print("="*70)

belief_slider.observe(show_recommendation, names='value')
display(belief_slider, output_recommendation)

# Initial display
show_recommendation(None)

## 1.5 Three Available Skills (from Neo4j)

In [ ]:
display(Markdown("""
### 🛠 Available Skills

Our agent has three skills to choose from. Let's query them from the database:
"""))

def query_crisp_skills():
    """Get crisp skill details from Neo4j"""
    if not NEO4J_CONNECTED:
        # Return hardcoded data if not connected
        return pd.DataFrame([
            {'Name': 'peek_door', 'Type': 'sense', 'Cost': 1.0, 
             'Goal Value': 0.0, 'Info Gain': 1.0,
             'Description': 'Look through keyhole'},
            {'Name': 'try_door', 'Type': 'act', 'Cost': 1.5,
             'Goal Value': 1.0, 'Info Gain': 0.0,
             'Description': 'Try to open door'},
            {'Name': 'go_window', 'Type': 'act', 'Cost': 2.0,
             'Goal Value': 1.0, 'Info Gain': 0.0,
             'Description': 'Escape via window'}
        ])
    
    results = run_query("""
        MATCH (s:Skill)
        WHERE s.kind IN ['sense', 'act'] 
          AND s.name IN ['peek_door', 'try_door', 'go_window']
        RETURN s.name as name, s.kind as kind, s.cost as cost,
               s.goal_info as goal, s.info_gain as info
        ORDER BY s.name
    """)
    
    if not results:
        print("⚠ No skills found in database. Using defaults.")
        return query_crisp_skills()  # Return defaults
    
    skills_data = []
    descriptions = {
        'peek_door': 'Look through keyhole',
        'try_door': 'Try to open door',
        'go_window': 'Escape via window'
    }
    
    for record in results:
        skills_data.append({
            'Name': record['name'],
            'Type': record['kind'],
            'Cost': record['cost'],
            'Goal Value': record['goal'],
            'Info Gain': record['info'],
            'Description': descriptions.get(record['name'], 'N/A')
        })
    
    return pd.DataFrame(skills_data)

skills_df = query_crisp_skills()

# Display table
display(skills_df.style.background_gradient(subset=['Cost'], cmap='Reds')
                        .background_gradient(subset=['Goal Value'], cmap='Greens')
                        .background_gradient(subset=['Info Gain'], cmap='Blues')
                        .format({'Cost': '{:.1f}', 'Goal Value': '{:.1f}', 'Info Gain': '{:.1f}'}))

display(Markdown("""
### 📊 Notice the Trade-offs:

**peek_door** 👁
- ✅ High info gain (1.0) - learns door state
- ✅ Low cost (1.0)
- ❌ Zero goal progress
- **Role**: Pure exploration

**try_door** 🚪
- ✅ High goal value (1.0) - might escape!
- ✅ Medium cost (1.5)
- ❌ Zero info gain
- ⚠ Only works if door unlocked
- **Role**: Pure exploitation (risky!)

**go_window** 🪟
- ✅ Guaranteed goal (1.0) - always escapes
- ❌ High cost (2.0) - risky!
- ❌ Zero info gain
- **Role**: Failsafe option

---

🤔 **The Question**: How should the agent choose between these?

That's what **Active Inference** solves! (Next section...)
"""))

## 1.6 Checkpoint 1: Test Your Intuition

In [ ]:
display(Markdown("""
### ✅ Checkpoint 1

**Scenario**: You believe the door is PROBABLY LOCKED (belief = 0.8)

**Question**: Which skill should you use first?
"""))

checkpoint1 = widgets.RadioButtons(
    options=['A) peek_door (gather info)',
             'B) try_door (act immediately)',
             'C) go_window (guaranteed escape)'],
    description='Your answer:',
    style={'description_width': 'initial'}
)

check1_button = widgets.Button(
    description="Check Answer",
    button_style='success',
    icon='check'
)

check1_output = widgets.Output()

def check_checkpoint1(b):
    with check1_output:
        clear_output()
        if 'peek_door' in checkpoint1.value:
            print("✓ Correct!\n")
            print("Reasoning:")
            print("  - If door is probably LOCKED (80% belief)...")
            print("  - Then trying it has 80% chance of failing (wasted effort)")
            print("  - Better to PEEK first (cheap, gives certainty)")
            print("  - Then decide: if unlocked→try, if locked→window")
            print("\n💡 This is the value of information!")
        elif 'try_door' in checkpoint1.value:
            print("✗ Not quite.\n")
            print("Think about it:")
            print("  - Door is probably LOCKED (80% belief)")
            print("  - So trying has 80% chance of failure")
            print("  - Cost = 1.5, but likely wasted")
            print("\n💡 Peeking first (cost=1.0) gives certainty before committing!")
        else:
            print("✗ Too cautious!\n")
            print("Think about it:")
            print("  - Window is expensive (cost=2.0) and risky")
            print("  - What if door is actually unlocked? (20% chance)")
            print("  - Even if locked, you'd want to check first")
            print("\n💡 Always explore door options before risky window escape!")
        
        print("\n" + "="*70)

check1_button.on_click(check_checkpoint1)
display(checkpoint1, check1_button, check1_output)

display(Markdown("""
---

### 🎯 Part 1 Summary

**What we learned**:
1. ✅ The locked room problem involves **uncertainty**
2. ✅ Available skills have **trade-offs** (cost, goal, info)
3. ✅ Your **beliefs** affect the best choice
4. ✅ Sometimes **gathering info first** is smarter than acting immediately

**Next**: How do we formalize this mathematically? → **Active Inference!**
"""))

---

# PART 2: Active Inference - The Math of Uncertainty

**Time**: 20 minutes

**Goals**:
- Understand Expected Free Energy (EFE) formula
- See how beliefs are probability distributions
- Calculate EFE for all three skills
- Visualize decision boundaries

**Now we formalize the intuition mathematically!**

## 2.1 Beliefs as Probability Distributions

In [ ]:
display(Markdown("""
### 📊 From Intuition to Mathematics

Previously, we talked about "belief" as a single number (0-1).

**More formally**: A belief is a **probability distribution** over possible states.

**In our case**:
- State space: {Unlocked, Locked}
- Belief: P(Locked) = b, P(Unlocked) = 1-b

Let's visualize how beliefs evolve:
"""))

# Interactive belief visualization
fig, ax = plt.subplots(figsize=(12, 5))

belief_param = widgets.FloatSlider(
    value=0.5, min=0.01, max=0.99, step=0.01,
    description='Belief (locked):',
    style={'description_width': 'initial'},
    continuous_update=True,
    readout_format='.0%'
)

output_belief_dist = widgets.Output()

def plot_belief(change):
    belief_locked = belief_param.value
    with output_belief_dist:
        clear_output(wait=True)
        
        states = ['Unlocked', 'Locked']
        probabilities = [1 - belief_locked, belief_locked]
        colors_viz = ['#2ecc71', '#e74c3c']
        
        fig, ax = plt.subplots(figsize=(10, 5))
        bars = ax.bar(states, probabilities, color=colors_viz, alpha=0.7, 
                     edgecolor='black', linewidth=2)
        ax.set_ylabel('Probability', fontsize=12)
        ax.set_title(f'Agent Belief Distribution (Locked={belief_locked:.1%})', 
                    fontsize=14, fontweight='bold')
        ax.set_ylim(0, 1.1)
        ax.grid(axis='y', alpha=0.3)
        
        # Add probability labels
        for bar, prob in zip(bars, probabilities):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                   f'{prob:.1%}', ha='center', va='bottom', 
                   fontsize=14, fontweight='bold')
        
        # Uncertainty measure
        entropy = -(belief_locked * np.log2(belief_locked + 1e-10) + 
                   (1-belief_locked) * np.log2(1-belief_locked + 1e-10))
        
        ax.text(0.5, 0.95, f'Uncertainty (entropy): {entropy:.3f} bits',
               ha='center', va='top', transform=ax.transAxes,
               bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
               fontsize=11)
        
        plt.tight_layout()
        plt.show()
        
        # Interpretation
        if entropy > 0.9:
            print("\n⚠ HIGH UNCERTAINTY - You really don't know!")
        elif entropy < 0.5:
            print("\n✓ LOW UNCERTAINTY - You're fairly confident!")
        else:
            print("\n~ MODERATE UNCERTAINTY")

belief_param.observe(plot_belief, names='value')
display(belief_param, output_belief_dist)
plot_belief(None)  # Initial display

## 2.2 Expected Free Energy (EFE) Formula

In [ ]:
display(Markdown(r"""
### 🧮 The Core Formula

Active inference agents choose actions by minimizing **Expected Free Energy** (G):

$$G(\pi) = \underbrace{\mathbb{E}[\text{cost}]}_{\text{Resource penalty}} - \underbrace{\mathbb{E}[\text{goal}]}_{\text{Pragmatic value}} - \underbrace{\mathbb{E}[\text{info}]}_{\text{Epistemic value}}$$

### Breaking It Down:

**1. Expected Cost** (Resources consumed)
- Time, energy, risk
- **Higher cost = worse** → Add to G (penalty)
- Example: window costs 2.0 (risky jump)

**2. Expected Goal** (How much does this help achieve objective?)
- Probability of escaping
- **Higher goal = better** → Subtract from G (reward)
- Example: try_door gives goal=1.0 IF door unlocked
- So expected_goal = 1.0 × P(Unlocked)

**3. Expected Info** (How much uncertainty does this reduce?)
- Information gain (reduction in entropy)
- **Higher info = better** → Subtract from G (reward)
- Example: peek_door gives info=1.0 (learns state)

### The Rule:
**Choose skill with LOWEST G** (best trade-off)

---

💡 **Key Insight**: This formula naturally balances:
- Exploration (info gain)
- Exploitation (goal achievement)  
- Efficiency (minimizing cost)

No separate "epsilon-greedy" or "UCB" needed!
"""))

## 2.3 Interactive EFE Calculator

In [ ]:
display(Markdown("""
### 🧪 Play with the Formula

Adjust the parameters and see how EFE changes:
"""))

cost_slider = widgets.FloatSlider(
    value=1.0, min=0, max=3, step=0.1, 
    description='Cost:', style={'description_width': 'initial'})
goal_slider = widgets.FloatSlider(
    value=0.5, min=0, max=1, step=0.05, 
    description='Goal Value:', style={'description_width': 'initial'})
info_slider = widgets.FloatSlider(
    value=0.5, min=0, max=1, step=0.05, 
    description='Info Gain:', style={'description_width': 'initial'})

output_efe = widgets.Output()

def calculate_efe(change):
    cost = cost_slider.value
    goal = goal_slider.value
    info = info_slider.value
    
    with output_efe:
        clear_output(wait=True)
        
        efe = cost - goal - info
        
        print("="*60)
        print("Expected Free Energy Calculation:")
        print("="*60)
        print(f"  Cost (penalty):    +{cost:.2f}")
        print(f"  Goal (reward):     -{goal:.2f}")
        print(f"  Info (reward):     -{info:.2f}")
        print("-" * 60)
        print(f"  EFE = {cost:.2f} - {goal:.2f} - {info:.2f} = {efe:.2f}")
        print("="*60)
        
        if efe < 0:
            print("\n✓✓ EXCELLENT skill (negative EFE = high net benefit!)")
        elif efe < 0.5:
            print("\n✓ GOOD skill (low EFE = decent trade-off)")
        elif efe < 1.5:
            print("\n~ OKAY skill (moderate EFE)")
        else:
            print("\n✗ POOR skill (high EFE = bad trade-off)")
        
        print("\n💡 Remember: LOWER EFE = BETTER choice!")

# Observe all sliders
cost_slider.observe(calculate_efe, names='value')
goal_slider.observe(calculate_efe, names='value')
info_slider.observe(calculate_efe, names='value')

display(widgets.VBox([cost_slider, goal_slider, info_slider]))
display(output_efe)
calculate_efe(None)  # Initial calculation

## 2.4 Code Implementation: score_skill()

In [ ]:
display(Markdown("""
### 💻 How Skills Are Scored in Code

Let's look at the actual implementation:
"""))

# Show the code
print("""
def score_skill(skill: Dict, belief_door_locked: float) -> float:
    '''
    Calculate Expected Free Energy for a skill.
    
    Args:
        skill: Dict with 'cost', 'goal_info', 'info_gain'
        belief_door_locked: Current belief that door is locked (0-1)
    
    Returns:
        Expected Free Energy (lower is better)
    '''
    # 1. Cost (always a penalty)
    cost = skill['cost']
    
    # 2. Expected goal (belief-weighted)
    # If door is probably unlocked, trying door has high expected goal
    # If door is probably locked, trying door has low expected goal
    belief_unlocked = 1 - belief_door_locked
    expected_goal = skill['goal_info'] * belief_unlocked
    
    # 3. Expected info (fixed per skill)
    # Sensing skills provide info, action skills don't
    expected_info = skill['info_gain']
    
    # 4. Calculate EFE
    efe = cost - expected_goal - expected_info
    
    return efe
""")

display(Markdown("""
### 🔍 Key Points:

1. **Cost** is always added (penalty)
2. **Goal** is belief-weighted:
   - If door probably unlocked → high expected_goal for try_door
   - If door probably locked → low expected_goal for try_door
3. **Info** is fixed per skill:
   - peek_door always gives info=1.0
   - try_door never gives info (info=0.0)
4. **Lower EFE wins** - agent picks skill with minimum G

This belief-weighting is why the optimal choice changes based on your belief!
"""))

## 2.5 Interactive: Score All Three Skills

In [ ]:
display(Markdown("""
### 🎯 Apply EFE Formula to Our Skills

Let's score all three skills (peek, try, window) across different belief values:
"""))

# Get skills data
skills_for_scoring = [
    {'name': 'peek_door', 'cost': 1.0, 'goal_info': 0.0, 'info_gain': 1.0},
    {'name': 'try_door', 'cost': 1.5, 'goal_info': 1.0, 'info_gain': 0.0},
    {'name': 'go_window', 'cost': 2.0, 'goal_info': 1.0, 'info_gain': 0.0}
]

# Interactive belief slider
belief_slider_scoring = widgets.FloatSlider(
    value=0.5, min=0, max=1, step=0.05,
    description='Belief (locked):',
    style={'description_width': 'initial'},
    continuous_update=True,
    readout_format='.0%'
)

output_scores = widgets.Output()

def score_all_skills(change):
    belief = belief_slider_scoring.value
    with output_scores:
        clear_output(wait=True)
        
        print("="*70)
        print(f"Belief: {belief:.0%} LOCKED, {1-belief:.0%} UNLOCKED")
        print("="*70)
        print("\nSkill Scoring:")
        print("-"*70)
        
        scores = {}
        for skill in skills_for_scoring:
            efe = score_skill(skill, belief)
            scores[skill['name']] = efe
            
            expected_goal = skill['goal_info'] * (1 - belief)
            
            print(f"\n{skill['name']:15s}:")
            print(f"  Cost = {skill['cost']:.2f}")
            print(f"  Goal = {skill['goal_info']:.2f} × {1-belief:.2f} = {expected_goal:.2f}")
            print(f"  Info = {skill['info_gain']:.2f}")
            print(f"  → EFE = {efe:.2f}")
        
        print("-"*70)
        best_skill = min(scores, key=scores.get)
        print(f"\n🎯 BEST CHOICE: {best_skill} (EFE = {scores[best_skill]:.2f})")
        print("="*70)

belief_slider_scoring.observe(score_all_skills, names='value')
display(belief_slider_scoring, output_scores)
score_all_skills(None)  # Initial

## 2.6 Visualization: EFE Curves Across Belief Space

In [ ]:
display(Markdown("""
### 📈 Decision Boundaries

Let's plot how EFE changes for each skill as beliefs change:
"""))

# Plot EFE curves
beliefs = np.linspace(0, 1, 100)

fig, ax = plt.subplots(figsize=(14, 7))

for skill in skills_for_scoring:
    efes = []
    for b in beliefs:
        efe = score_skill(skill, b)
        efes.append(efe)
    
    ax.plot(beliefs, efes, label=skill['name'], linewidth=3, alpha=0.8)

ax.set_xlabel('Belief that door is LOCKED', fontsize=13, fontweight='bold')
ax.set_ylabel('Expected Free Energy (lower = better)', fontsize=13, fontweight='bold')
ax.set_title('Skill Selection Depends on Belief!', fontsize=16, fontweight='bold', pad=15)
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5, label='EFE = 0')
ax.grid(alpha=0.3)
ax.legend(fontsize=12, loc='best')
ax.invert_yaxis()  # Lower is better, so show best at top

# Mark crossover points
ax.axvline(x=0.5, color='red', linestyle=':', alpha=0.5, label='Decision boundary')

plt.tight_layout()
plt.show()

display(Markdown("""
### 🔍 What This Shows:

**peek_door** (blue):
- Constant EFE = 0.0 (cost=1, info=1, goal=0)
- Always provides same value regardless of belief

**try_door** (orange):
- EFE increases as belief (locked) increases
- When belief < ~0.5: try_door is best (probably unlocked)
- When belief > ~0.5: peek_door is better (too uncertain to try)

**go_window** (green):
- Constant EFE = 1.0 (cost=2, goal=1, info=0)
- Always worst option (expensive!)
- Only use as last resort

**Key Insight**: The **crossover points** show when optimal policy switches!

This is Active Inference in action - balancing exploration and exploitation naturally!
"""))

## 2.7 Checkpoint 2: Calculate EFE

In [ ]:
display(Markdown("""
### ✅ Checkpoint 2: Test Your Understanding

**Given**:
- Skill: try_door
- cost = 1.5
- goal_info = 1.0
- info_gain = 0.0
- belief (locked) = 0.3

**Calculate**: What is the EFE?
"""))

checkpoint2_answer = widgets.FloatText(
    value=0.0,
    description='Your EFE:',
    style={'description_width': 'initial'}
)

check2_button = widgets.Button(
    description="Check Answer",
    button_style='success'
)

check2_output = widgets.Output()

def check_checkpoint2(b):
    with check2_output:
        clear_output()
        
        user_answer = checkpoint2_answer.value
        
        # Correct calculation
        belief = 0.3
        cost = 1.5
        expected_goal = 1.0 * (1 - belief)  # 1.0 * 0.7 = 0.7
        expected_info = 0.0
        correct_efe = cost - expected_goal - expected_info  # 1.5 - 0.7 - 0 = 0.8
        
        if abs(user_answer - correct_efe) < 0.01:
            print("✓ Correct!\n")
            print(f"EFE = {correct_efe:.2f}")
            print("\nCalculation:")
            print(f"  Cost = {cost:.2f}")
            print(f"  Expected goal = {1.0:.2f} × {1-belief:.2f} = {expected_goal:.2f}")
            print(f"  Expected info = {expected_info:.2f}")
            print(f"  EFE = {cost:.2f} - {expected_goal:.2f} - {expected_info:.2f} = {correct_efe:.2f}")
        else:
            print(f"✗ Not quite. The correct EFE is {correct_efe:.2f}\n")
            print("Hint: Remember the formula:")
            print("  EFE = cost - (goal × P(unlocked)) - info")
            print(f"  EFE = {cost:.2f} - ({1.0:.2f} × {1-belief:.2f}) - {expected_info:.2f}")
            print(f"  EFE = {cost:.2f} - {expected_goal:.2f} - {expected_info:.2f}")
            print(f"  EFE = {correct_efe:.2f}")

check2_button.on_click(check_checkpoint2)
display(checkpoint2_answer, check2_button, check2_output)

---

### 🎯 Part 2 Summary

**What we learned**:

1. ✅ **Beliefs** = probability distributions over states
2. ✅ **Expected Free Energy** = cost - goal - info
3. ✅ **Lower EFE = better** choice
4. ✅ **Goal is belief-weighted** - that's why beliefs matter!
5. ✅ **Decision boundaries** emerge from EFE curves
6. ✅ **Active inference naturally balances** exploration and exploitation

**Next**: How do agents learn and adapt? → **Procedural Memory & Episodes!**

---

# PART 3: Policy Execution & Learning

**Time**: 25 minutes

**Goals**:
- Understand how beliefs UPDATE after observations
- See Bayesian inference in action
- Explore procedural memory networks
- Simulate a complete episode
- Visualize the knowledge graph

**The bridge**: From scoring skills → to understanding agent behavior

## 3.1 The Knowledge Graph: Database Schema

In [ ]:
display(Markdown("""
### 🗄️ What's in Our Neo4j Database?

Before we dive into execution, let's understand the graph structure:
"""))

if NEO4J_CONNECTED:
    # Get schema information
    schema_info = run_query("""
        CALL db.labels() YIELD label
        RETURN label, count{(n) WHERE label IN labels(n)} as count
        ORDER BY count DESC
    """)
    
    print("\n📊 Node Types in Database:")
    print("=" * 50)
    for item in schema_info:
        print(f"  {item['label']:20s}: {item['count']:3d} nodes")
    
    # Get relationship types
    rel_info = run_query("""
        CALL db.relationshipTypes() YIELD relationshipType
        RETURN relationshipType, 
               count{()-[r]-() WHERE type(r) = relationshipType} as count
        ORDER BY count DESC
    """)
    
    print("\n🔗 Relationship Types:")
    print("=" * 50)
    for item in rel_info:
        print(f"  {item['relationshipType']:20s}: {item['count']:3d} relationships")
    
    # Visualize schema
    fig, ax = plt.subplots(figsize=(14, 10))
    
    # Create schema graph
    G_schema = nx.DiGraph()
    
    # Define schema (conceptual)
    schema_nodes = {
        'Agent': {'color': '#3498db', 'pos': (0, 3)},
        'State': {'color': '#e74c3c', 'pos': (-2, 1)},
        'Skill': {'color': '#2ecc71', 'pos': (2, 1)},
        'Observation': {'color': '#f39c12', 'pos': (-2, -1)},
        'Memory': {'color': '#9b59b6', 'pos': (2, -1)}
    }
    
    schema_edges = [
        ('Agent', 'State', 'IN_STATE'),
        ('Agent', 'Skill', 'CAN_USE'),
        ('State', 'State', 'LEADS_TO'),
        ('State', 'Observation', 'YIELDS'),
        ('Skill', 'Observation', 'PRODUCES'),
        ('Memory', 'Skill', 'RECOMMENDS')
    ]
    
    for node, attrs in schema_nodes.items():
        G_schema.add_node(node, **attrs)
    
    for src, dst, label in schema_edges:
        G_schema.add_edge(src, dst, label=label)
    
    # Position nodes
    pos = {node: attrs['pos'] for node, attrs in schema_nodes.items()}
    
    # Draw
    node_colors = [schema_nodes[node]['color'] for node in G_schema.nodes()]
    nx.draw_networkx_nodes(G_schema, pos, node_color=node_colors,
                          node_size=3000, alpha=0.9, edgecolors='black', linewidths=2)
    nx.draw_networkx_labels(G_schema, pos, font_size=12, font_weight='bold')
    
    # Draw edges with labels
    nx.draw_networkx_edges(G_schema, pos, width=2, alpha=0.6,
                          arrows=True, arrowsize=20, arrowstyle='-|>',
                          connectionstyle='arc3,rad=0.1')
    
    edge_labels = {(e[0], e[1]): e[2] for e in schema_edges}
    nx.draw_networkx_edge_labels(G_schema, pos, edge_labels, font_size=9)
    
    ax.set_title('MacGyver MUD Knowledge Graph Schema', 
                fontsize=16, fontweight='bold', pad=20)
    ax.axis('off')
    plt.tight_layout()
    plt.show()
    
else:
    print("\n⚠ Neo4j not connected. Showing conceptual schema.")
    print("\nKey Node Types:")
    print("  - Agent: The decision-maker")
    print("  - State: Room states (stuck, escaped, etc.)")
    print("  - Skill: Available actions (peek, try, window)")
    print("  - Observation: What the agent perceives")
    print("  - Memory: Learned patterns (context → skill)")

display(Markdown("""
### 📖 Understanding the Schema

**Nodes (Entities)**:
- 🤖 **Agent**: The decision-making entity (MacGyverBot)
- 🚪 **State**: Possible world states (stuck_in_room, escaped, etc.)
- 🛠 **Skill**: Available actions (peek_door, try_door, go_window)
- 👁 **Observation**: What the agent perceives after actions
- 🧠 **Memory**: Learned associations (context → recommended skill)

**Relationships (Connections)**:
- `CAN_USE`: Agent can use certain skills
- `LEADS_TO`: State transitions (stuck → escaped)
- `PRODUCES`: Skills produce observations
- `RECOMMENDS`: Memories recommend skills for contexts
- `YIELDS`: States can yield observations

This graph structure enables:
1. **Planning**: Find paths from stuck → escaped
2. **Learning**: Store context → skill patterns
3. **Reasoning**: Query relationships and patterns
"""))

## 3.2 How Beliefs Update: Bayesian Inference

In [ ]:
display(Markdown(r"""
### 🔄 From Uncertainty to Knowledge

When the agent executes `peek_door`, it observes something. How does this update beliefs?

**Bayes' Rule**:

$$P(\text{Locked} | \text{observation}) = \frac{P(\text{observation} | \text{Locked}) \cdot P(\text{Locked})}{P(\text{observation})}$$

**In English**:
- **Prior**: P(Locked) = initial belief before peeking
- **Likelihood**: P(obs|Locked) = how likely this observation if door is locked
- **Posterior**: P(Locked|obs) = updated belief after observation

**Example**:
- Prior: 50% chance door is locked
- Action: peek_door
- Observation: "obs_door_locked"
- Likelihood: If locked, 95% chance we observe "locked" (sensing accuracy)
- Posterior: ~95% chance door is locked (updated!)

Let's see this in action!
"""))

# Interactive belief update simulator
def simulate_belief_update(prior_locked, true_state, action):
    """
    Simulate one belief update step.
    
    Args:
        prior_locked: Prior belief that door is locked (0-1)
        true_state: True door state ('locked' or 'unlocked')
        action: Action taken ('peek_door', 'try_door')
    
    Returns:
        posterior_locked, observation
    """
    # Sensing accuracy
    accuracy = 0.95
    
    if action == 'peek_door':
        # Generate observation based on true state
        if true_state == 'locked':
            obs = 'obs_door_locked' if np.random.random() < accuracy else 'obs_door_unlocked'
        else:
            obs = 'obs_door_unlocked' if np.random.random() < accuracy else 'obs_door_locked'
        
        # Bayesian update
        if obs == 'obs_door_locked':
            # P(Locked|obs_locked) using Bayes
            likelihood_locked = accuracy
            likelihood_unlocked = 1 - accuracy
            
            numerator = likelihood_locked * prior_locked
            denominator = (likelihood_locked * prior_locked + 
                          likelihood_unlocked * (1 - prior_locked))
            posterior_locked = numerator / denominator
        else:
            # P(Locked|obs_unlocked)
            likelihood_locked = 1 - accuracy
            likelihood_unlocked = accuracy
            
            numerator = likelihood_locked * prior_locked
            denominator = (likelihood_locked * prior_locked + 
                          likelihood_unlocked * (1 - prior_locked))
            posterior_locked = numerator / denominator
    
    elif action == 'try_door':
        # Try door gives indirect evidence
        if true_state == 'locked':
            obs = 'obs_door_still_locked'
            posterior_locked = 0.99  # Very confident it's locked
        else:
            obs = 'obs_escaped'
            posterior_locked = 0.0  # Certain it was unlocked!
    else:
        obs = 'no_observation'
        posterior_locked = prior_locked
    
    return posterior_locked, obs

# Interactive widget
prior_slider = widgets.FloatSlider(
    value=0.5, min=0.01, max=0.99, step=0.01,
    description='Prior belief (locked):',
    style={'description_width': 'initial'},
    readout_format='.0%'
)

true_state_radio = widgets.RadioButtons(
    options=['locked', 'unlocked'],
    value='locked',
    description='True state:',
    style={'description_width': 'initial'}
)

action_radio = widgets.RadioButtons(
    options=['peek_door', 'try_door'],
    value='peek_door',
    description='Action:',
    style={'description_width': 'initial'}
)

update_button = widgets.Button(
    description='Execute Action',
    button_style='primary',
    icon='play'
)

update_output = widgets.Output()

def on_update_click(b):
    with update_output:
        clear_output(wait=True)
        
        prior = prior_slider.value
        true_state = true_state_radio.value
        action = action_radio.value
        
        posterior, obs = simulate_belief_update(prior, true_state, action)
        
        # Visualize
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 5))
        
        # Prior
        ax1.bar(['Unlocked', 'Locked'], [1-prior, prior], 
               color=['#2ecc71', '#e74c3c'], alpha=0.7, edgecolor='black', linewidth=2)
        ax1.set_ylim(0, 1.1)
        ax1.set_ylabel('Probability', fontsize=12)
        ax1.set_title('BEFORE: Prior Belief', fontsize=14, fontweight='bold')
        ax1.text(0, 1-prior+0.02, f'{1-prior:.0%}', ha='center', fontsize=14, fontweight='bold')
        ax1.text(1, prior+0.02, f'{prior:.0%}', ha='center', fontsize=14, fontweight='bold')
        ax1.grid(axis='y', alpha=0.3)
        
        # Action
        ax2.axis('off')
        ax2.text(0.5, 0.7, f'Action: {action}', ha='center', va='center',
                fontsize=16, fontweight='bold',
                bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.7))
        ax2.text(0.5, 0.5, f'Observation:\n{obs}', ha='center', va='center',
                fontsize=14, style='italic',
                bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.7))
        ax2.text(0.5, 0.3, f'True state: {true_state}\n(hidden from agent)', 
                ha='center', va='center', fontsize=11, color='gray')
        ax2.set_xlim(0, 1)
        ax2.set_ylim(0, 1)
        
        # Posterior
        ax3.bar(['Unlocked', 'Locked'], [1-posterior, posterior],
               color=['#2ecc71', '#e74c3c'], alpha=0.7, edgecolor='black', linewidth=2)
        ax3.set_ylim(0, 1.1)
        ax3.set_ylabel('Probability', fontsize=12)
        ax3.set_title('AFTER: Posterior Belief', fontsize=14, fontweight='bold')
        ax3.text(0, 1-posterior+0.02, f'{1-posterior:.0%}', ha='center', fontsize=14, fontweight='bold')
        ax3.text(1, posterior+0.02, f'{posterior:.0%}', ha='center', fontsize=14, fontweight='bold')
        ax3.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Summary
        print("\n" + "="*70)
        print("BAYESIAN UPDATE SUMMARY")
        print("="*70)
        print(f"Prior belief (Locked):     {prior:.2%}")
        print(f"Action taken:              {action}")
        print(f"Observation received:      {obs}")
        print(f"Posterior belief (Locked): {posterior:.2%}")
        print("-"*70)
        change = posterior - prior
        print(f"Belief change:             {change:+.2%}")
        print(f"Information gained:        {abs(change):.2%}")
        print("="*70)
        
        if abs(change) > 0.3:
            print("\n✓ Significant information gained!")
        elif abs(change) > 0.1:
            print("\n~ Moderate information gained.")
        else:
            print("\n⚠ Minimal information gained (already confident).")

update_button.on_click(on_update_click)

display(Markdown("""
### 🎮 Interactive Belief Update Simulator

Try different scenarios:
1. Prior=50%, True=locked, Action=peek → See confidence increase
2. Prior=80%, True=unlocked, Action=peek → See surprise! (belief reversal)
3. Prior=50%, True=unlocked, Action=try_door → Immediate certainty!
"""))

display(widgets.VBox([
    prior_slider,
    true_state_radio,
    action_radio,
    update_button
]))
display(update_output)

## 3.3 Procedural Memory: Learning from Experience

In [ ]:
display(Markdown("""
### 🧠 How the Agent Learns Patterns

**Procedural Memory** = Learned associations between contexts and skills

**Structure**: `(context_pattern) → recommended_skill (confidence)`

**Example Memories**:
- "When belief > 0.7 AND tried=false → peek_door" (confidence: 0.9)
- "When tried=true AND failed → go_window" (confidence: 0.8)

**How it works**:
1. Agent executes episodes
2. Successful paths remembered
3. Context patterns extracted
4. Future decisions influenced by memories

Let's visualize the memory network!
"""))

if NEO4J_CONNECTED:
    # Query procedural memories
    memories = run_query("""
        MATCH (m:Memory)-[r:RECOMMENDS]->(s:Skill)
        RETURN m.context as context, 
               s.name as skill, 
               r.confidence as confidence
        ORDER BY r.confidence DESC
        LIMIT 10
    """)
    
    if memories:
        print("\n📚 Top 10 Procedural Memories:")
        print("=" * 70)
        for i, mem in enumerate(memories, 1):
            print(f"\n{i}. Context: {mem['context']}")
            print(f"   → Recommends: {mem['skill']}")
            print(f"   → Confidence: {mem['confidence']:.2f}")
        
        # Visualize memory network
        fig, ax = plt.subplots(figsize=(14, 10))
        G_mem = nx.DiGraph()
        
        for mem in memories:
            context_node = mem['context'][:30] + '...' if len(mem['context']) > 30 else mem['context']
            G_mem.add_edge(context_node, mem['skill'], 
                          weight=mem['confidence'])
        
        pos = nx.spring_layout(G_mem, k=2, iterations=50)
        
        # Color by node type
        node_colors = ['#9b59b6' if 'belief' in node.lower() or 'tried' in node.lower() 
                      else '#2ecc71' for node in G_mem.nodes()]
        
        nx.draw(G_mem, pos, with_labels=True, node_color=node_colors,
               node_size=2000, font_size=9, font_weight='bold',
               arrows=True, edge_color='gray', width=2, alpha=0.7,
               arrowsize=15)
        
        # Add edge labels (confidence)
        edge_labels = {(u, v): f"{d['weight']:.2f}" 
                      for u, v, d in G_mem.edges(data=True)}
        nx.draw_networkx_edge_labels(G_mem, pos, edge_labels, font_size=8)
        
        ax.set_title('Procedural Memory Network (Context → Skill)', 
                    fontsize=16, fontweight='bold', pad=20)
        ax.axis('off')
        plt.tight_layout()
        plt.show()
    else:
        print("\n⚠ No procedural memories found in database.")
        print("   (Memories are created during episodes with --use-memory flag)")
else:
    print("\n⚠ Neo4j not connected. Can't query memories.")

display(Markdown("""
### 💡 How Memories Improve Decision-Making

**Without Memory** (Pure EFE):
- Calculate EFE for each skill
- Pick minimum
- No learning across episodes

**With Memory** (EFE + Experience):
- Calculate EFE
- Check if context matches any memory
- Boost confidence for recommended skills
- Faster convergence to good strategies

**Example**:
- Context: "belief=0.8, tried=false"
- Memory says: "peek_door worked well here before"
- Agent more likely to peek (even if EFE is similar to try)

This is **procedural learning** - learning "what works when"!
"""))

## 3.4 Simulating a Complete Episode

In [ ]:
display(Markdown("""
### 🎬 Watch an Agent Episode

Let's simulate a complete episode step-by-step:
"""))

def simulate_episode(initial_belief, true_door_state, max_steps=5):
    """
    Simulate a complete episode.
    """
    belief = initial_belief
    escaped = False
    step = 0
    history = []
    
    skills_available = [
        {'name': 'peek_door', 'cost': 1.0, 'goal_info': 0.0, 'info_gain': 1.0},
        {'name': 'try_door', 'cost': 1.5, 'goal_info': 1.0, 'info_gain': 0.0},
        {'name': 'go_window', 'cost': 2.0, 'goal_info': 1.0, 'info_gain': 0.0}
    ]
    
    print("\n" + "="*70)
    print(f"EPISODE SIMULATION (True state: {true_door_state})")
    print("="*70)
    print(f"\nInitial belief (locked): {belief:.2%}")
    print()
    
    while not escaped and step < max_steps:
        step += 1
        print(f"\n--- STEP {step} ---")
        print(f"Current belief (locked): {belief:.2%}")
        
        # Score all skills
        scores = {}
        for skill in skills_available:
            efe = score_skill(skill, belief)
            scores[skill['name']] = efe
            print(f"  {skill['name']:15s}: EFE = {efe:.2f}")
        
        # Choose best skill
        chosen_skill_name = min(scores, key=scores.get)
        chosen_skill = next(s for s in skills_available if s['name'] == chosen_skill_name)
        print(f"\n→ Chosen: {chosen_skill_name} (lowest EFE)")
        
        # Execute and update belief
        if chosen_skill_name == 'peek_door':
            new_belief, obs = simulate_belief_update(belief, true_door_state, 'peek_door')
            print(f"  Observation: {obs}")
            print(f"  Updated belief (locked): {new_belief:.2%}")
            belief = new_belief
        
        elif chosen_skill_name == 'try_door':
            if true_door_state == 'unlocked':
                print(f"  Success! Door was unlocked. ESCAPED!")
                escaped = True
            else:
                print(f"  Failed. Door is locked.")
                belief = 0.99  # Now very confident it's locked
        
        elif chosen_skill_name == 'go_window':
            print(f"  Escaped via window (costly but guaranteed).")
            escaped = True
        
        history.append({
            'step': step,
            'belief': belief,
            'skill': chosen_skill_name,
            'efe': scores[chosen_skill_name]
        })
    
    print("\n" + "="*70)
    if escaped:
        print(f"✓ ESCAPED in {step} steps!")
    else:
        print(f"✗ Did not escape within {max_steps} steps.")
    print("="*70)
    
    return history

# Run simulation
print("\n🎮 Scenario 1: Door is actually UNLOCKED")
history1 = simulate_episode(initial_belief=0.5, true_door_state='unlocked')

print("\n\n🎮 Scenario 2: Door is actually LOCKED")
history2 = simulate_episode(initial_belief=0.5, true_door_state='locked')

display(Markdown("""
### 🎯 Observe the Strategy

**Scenario 1 (Unlocked)**:
- Agent likely tries door early (belief ~0.5)
- Succeeds immediately!
- Fast escape

**Scenario 2 (Locked)**:
- Agent may peek first (uncertain)
- Learns door is locked
- Switches to window

**Key Insight**: Same algorithm, different outcomes based on:
1. Initial beliefs
2. True world state
3. Observations received

This is **active inference** adapting to the environment!
"""))

## 3.5 Checkpoint 3: Understanding Belief Updates

In [ ]:
display(Markdown("""
### ✅ Checkpoint 3

**Scenario**:
- Prior belief (locked): 60%
- Action: peek_door
- Observation: obs_door_unlocked
- Sensing accuracy: 95%

**Question**: What should the posterior belief (locked) be approximately?

*Think about it: If you thought 60% locked, but observed "unlocked" with 95% accuracy...*
"""))

checkpoint3_answer = widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1,
    step=0.05,
    description='Posterior (locked):',
    style={'description_width': 'initial'},
    readout_format='.0%'
)

check3_button = widgets.Button(
    description="Check Answer",
    button_style='success'
)

check3_output = widgets.Output()

def check_checkpoint3(b):
    with check3_output:
        clear_output()
        
        user_answer = checkpoint3_answer.value
        
        # Correct Bayesian calculation
        prior = 0.6
        accuracy = 0.95
        
        # P(Locked | obs_unlocked)
        likelihood_locked = 1 - accuracy  # 0.05
        likelihood_unlocked = accuracy     # 0.95
        
        numerator = likelihood_locked * prior
        denominator = (likelihood_locked * prior + 
                      likelihood_unlocked * (1 - prior))
        correct = numerator / denominator
        
        if abs(user_answer - correct) < 0.1:
            print("✓ Correct! (or very close)\n")
            print(f"Exact answer: {correct:.1%}")
            print("\nExplanation:")
            print(f"  Prior: {prior:.0%} locked")
            print(f"  Observed: 'unlocked' (95% accurate sensing)")
            print(f"  This is STRONG evidence door is unlocked!")
            print(f"  Posterior: {correct:.1%} locked (confidence flipped!)")
        else:
            print(f"✗ Not quite. Correct answer: {correct:.1%}\n")
            print("Reasoning:")
            print(f"  You saw 'unlocked' with 95% accuracy")
            print(f"  Prior was only 60% locked (moderate)")
            print(f"  Strong evidence → belief should flip!")
            print(f"  New belief: {correct:.1%} locked = {1-correct:.1%} unlocked")

check3_button.on_click(check_checkpoint3)
display(checkpoint3_answer, check3_button, check3_output)

---

### 🎯 Part 3 Summary

**What we learned**:

1. ✅ **Database schema** - Node types and relationships in Neo4j
2. ✅ **Bayesian inference** - How beliefs update after observations
3. ✅ **Belief dynamics** - Prior → Observation → Posterior
4. ✅ **Procedural memory** - Learning context → skill patterns
5. ✅ **Episode simulation** - Complete agent execution from start to escape
6. ✅ **Adaptive behavior** - Same algorithm, different strategies based on observations

### 🔗 The Bridge

We've now seen:
- **Part 1**: The problem (locked door)
- **Part 2**: The math (EFE formula)
- **Part 3**: The execution (beliefs update, agent learns)

**Next**: Can we UNDERSTAND the agent's strategy at a glance?

→ **Part 4: The Silver Gauge** (geometric diagnostics!)

We need interpretable metrics that don't change behavior...

Enter: Pythagorean means! 📐

---

# PART 4: The Silver Gauge - Geometric Fingerprinting

**Time**: 25 minutes

**Goals**:
- Understand Pythagorean means (HM, GM, AM)
- Create dimensionless k coefficients
- Apply to our skills
- **DISCOVER**: Why k ≈ 0 for ALL crisp skills!
- Reveal the geometric gap

⭐ **This is the climax** - the big revelation!

## 4.1 The Interpretability Challenge

In [ ]:
display(Markdown("""
### 🤔 The Problem with EFE

We've built an active inference agent that:
- ✅ Balances exploration and exploitation
- ✅ Updates beliefs using observations
- ✅ Chooses optimal skills

**But**: Can we UNDERSTAND its strategy at a glance?

**Challenge**: Create interpretable metrics WITHOUT changing behavior
- Requirement: 100% behavioral fidelity (no approximation)
- Goal: Geometric \"fingerprint\" of decision-making style
- Must be: Scale-invariant, dimensionless, bounded

**Enter**: The Silver Gauge

### 💡 Core Idea

Instead of just calculating EFE, let's create \"shape coefficients\" that reveal:
- How balanced is this skill? (specialist vs generalist)
- How efficient is this skill? (benefit vs cost)

**Method**: Pythagorean means (from ~500 BCE!)
"""))

## 4.2 Pythagorean Means: 2500-Year-Old Math

In [ ]:
display(Markdown(r"""
### 📐 Three Classical Means

Given two positive numbers $a$ and $b$, the ancient Greeks defined three averages:

#### 1. Harmonic Mean (HM) - The Bottleneck Penalizer

$$HM = \frac{2ab}{a + b}$$

- Severely penalizes imbalance
- If either $a$ or $b$ is small, HM is small
- Used for: rates, speeds, efficiency ratios
- Example: Average speed for round trip

#### 2. Geometric Mean (GM) - The Balanced Multiplier

$$GM = \sqrt{ab}$$

- Respects proportional relationships
- Balanced compromise between values
- Used for: growth rates, aspect ratios
- Example: Average percentage growth

#### 3. Arithmetic Mean (AM) - The Fair Splitter

$$AM = \frac{a + b}{2}$$

- Simple average (what you learned in school)
- Fair split of sum
- Used for: central tendency
- Example: Average height

---

### 🎓 The Pythagorean Inequality

**Always true**: $HM \leq GM \leq AM$

**Equality only when**: $a = b$ (perfect balance)

Let's explore this interactively!
"""))

## 4.3 Interactive: Explore Pythagorean Means

In [ ]:
display(Markdown("""
### 🧮 Pythagorean Means Calculator

Try different values and watch the means!
"""))

a_slider = widgets.FloatSlider(value=2, min=0.1, max=10, step=0.1, description='a:')
b_slider = widgets.FloatSlider(value=8, min=0.1, max=10, step=0.1, description='b:')

means_output = widgets.Output()

def calculate_means(change):
    a = a_slider.value
    b = b_slider.value
    
    with means_output:
        clear_output(wait=True)
        
        # Calculate means
        hm = (2 * a * b) / (a + b)
        gm = np.sqrt(a * b)
        am = (a + b) / 2
        
        # Balance ratio
        k = gm / am
        
        # Visualize
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Left: Bar chart of means
        means_values = [hm, gm, am]
        means_names = ['HM\n(bottleneck)', 'GM\n(balanced)', 'AM\n(fair split)']
        colors_means = ['#e74c3c', '#f39c12', '#3498db']
        
        bars = ax1.bar(means_names, means_values, color=colors_means, 
                      alpha=0.7, edgecolor='black', linewidth=2)
        ax1.set_ylabel('Value', fontsize=12)
        ax1.set_title(f'Pythagorean Means for a={a:.1f}, b={b:.1f}', 
                     fontsize=14, fontweight='bold')
        ax1.grid(axis='y', alpha=0.3)
        
        # Add value labels
        for bar, val in zip(bars, means_values):
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height,
                    f'{val:.3f}', ha='center', va='bottom', 
                    fontsize=11, fontweight='bold')
        
        # Show inequality
        ax1.plot([0, 2], [hm, am], 'k--', alpha=0.3, linewidth=2)
        ax1.text(1, (hm + am)/2, f'HM ≤ GM ≤ AM', 
                ha='center', fontsize=10, style='italic',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        # Right: Input values
        ax2.barh(['a', 'b'], [a, b], color=['#2ecc71', '#9b59b6'], 
                alpha=0.7, edgecolor='black', linewidth=2)
        ax2.set_xlabel('Value', fontsize=12)
        ax2.set_title('Input Values', fontsize=14, fontweight='bold')
        ax2.grid(axis='x', alpha=0.3)
        
        # Balance indicator
        balance_ratio = min(a, b) / max(a, b)
        ax2.text(0.5, 0.1, f'Balance: {balance_ratio:.1%}\n(1.0 = perfect)',
                transform=ax2.transAxes, ha='center',
                bbox=dict(boxstyle='round', 
                         facecolor='lightgreen' if balance_ratio > 0.7 else 'lightyellow',
                         alpha=0.7))
        
        plt.tight_layout()
        plt.show()
        
        # Text output
        print("="*70)
        print(f"Input: a={a:.2f}, b={b:.2f}")
        print("="*70)
        print(f"Harmonic Mean (HM):  {hm:.4f}  ← Bottleneck penalizer")
        print(f"Geometric Mean (GM): {gm:.4f}  ← Balanced multiplier")
        print(f"Arithmetic Mean (AM):{am:.4f}  ← Fair splitter")
        print("-"*70)
        print(f"Verification: {hm:.4f} ≤ {gm:.4f} ≤ {am:.4f}")
        print(f"Inequality holds: {hm <= gm <= am}")
        print("="*70)
        print(f"\nBalance ratio k = GM/AM = {k:.4f}")
        if k > 0.95:
            print("  → Nearly perfect balance! (a ≈ b)")
        elif k > 0.7:
            print("  → Good balance")
        elif k > 0.5:
            print("  → Moderate imbalance")
        else:
            print("  → Severe imbalance (specialist!)")

a_slider.observe(calculate_means, names='value')
b_slider.observe(calculate_means, names='value')

display(a_slider, b_slider, means_output)
calculate_means(None)  # Initial

display(Markdown("""
### 💡 Experiments to Try:

1. **Perfect balance**: Set a=5, b=5
   - Notice: HM = GM = AM (all equal!)
   - k = 1.0

2. **Moderate imbalance**: Set a=3, b=7
   - Notice: HM < GM < AM (inequality holds)
   - k ≈ 0.93 (still pretty balanced)

3. **Severe imbalance**: Set a=0.1, b=10
   - Notice: HM is VERY small (bottleneck!)
   - k ≈ 0.20 (specialist!)

**Key Insight**: The ratio k = GM/AM measures balance!
"""))

## 4.4 Creating Dimensionless Ratios: k Coefficients

In [ ]:
display(Markdown(r"""
### 🎯 From Means to Shape Coefficients

**Key Insight**: The RATIO $k = \frac{GM}{AM}$ is special:

#### Properties of k:

1. **Dimensionless**: No units (pure number)
   - Doesn't matter if inputs are seconds, dollars, or utilities

2. **Scale-invariant**: $k(2, 8) = k(20, 80) = k(200, 800)$
   - Only the RATIO matters, not absolute scale

3. **Bounded**: $k \in [0, 1]$
   - 0 = severe imbalance (specialist)
   - 1 = perfect balance (generalist)

4. **Interpretable**: Geometric meaning
   - k close to 1: values are similar
   - k close to 0: one value dominates

### Mathematical Formula:

$$k = \frac{GM}{AM} = \frac{\sqrt{ab}}{\frac{a+b}{2}} = \frac{2\sqrt{ab}}{a+b}$$

**When is k = 1?** When $a = b$ (perfect balance)

**When is k → 0?** When $a \ll b$ or $b \ll a$ (specialist)

---

### 💡 Why This Matters for AI

We can use k to create **scale-invariant metrics** that:
- Transfer across domains
- Are interpretable
- Don't change behavior (100% fidelity)

Let's apply this to our skills!
"""))

## 4.5 Silver Gauge Coefficient #1: k_explore

In [ ]:
display(Markdown(r"""
### 🔍 k_explore: Balance Between Goal and Info

**Question**: Does this skill balance goal achievement and information gain?

**Inputs**:
- $a$ = goal value (pragmatic component)
- $b$ = info gain (epistemic component)

**Calculation**:
$$k_{\text{explore}} = \frac{GM(\text{goal}, \text{info})}{AM(\text{goal}, \text{info})} = \frac{2\sqrt{\text{goal} \cdot \text{info}}}{\text{goal} + \text{info}}$$

**Interpretation**:
- $k_{\text{explore}} \approx 1$: **Balanced** multi-objective skill
- $k_{\text{explore}} \approx 0$: **Specialist** (pure exploration OR pure exploitation)

### ⚠️ Critical Insight

A **pure exploration** skill (info=100%, goal=0%) has $k \approx 0$!

A **pure exploitation** skill (goal=100%, info=0%) also has $k \approx 0$!

**Both are specialists** - just in opposite directions!

Only **multi-objective** skills (goal AND info both substantial) have $k > 0.5$!

---

### 💻 Code Implementation
"""))

print("""
def silver_k_explore(goal_value: float, info_gain: float) -> float:
    '''
    Calculate k_explore coefficient using Pythagorean means.
    
    Args:
        goal_value: Expected goal achievement [0, 1]
        info_gain: Expected information gain [0, 1]
    
    Returns:
        k_explore ∈ [0, 1] where 1 = perfect balance, 0 = specialist
    '''
    epsilon = 1e-10  # Avoid division by zero
    
    # Geometric mean: balanced multiplier
    gm = np.sqrt(goal_value * info_gain + epsilon)
    
    # Arithmetic mean: fair splitter
    am = (goal_value + info_gain) / 2.0 + epsilon
    
    # Dimensionless ratio
    k = gm / am
    
    return k
""")

display(Markdown("""
### ✅ Mathematical Proof of Behavioral Fidelity

**Claim**: Silver Gauge doesn't change which skill has minimum EFE

**Proof**:
1. EFE uses: `cost`, `goal_value`, `info_gain`
2. Silver Gauge uses: `goal_value`, `info_gain` (same inputs!)
3. Silver Gauge is a **pure function** - no side effects
4. Argmin(EFE) is unchanged

**Therefore**: 100% behavioral fidelity guaranteed!

We've added a diagnostic layer without changing the model!
"""))

## 4.6 🎉 THE REVELATION: Calculate k_explore for Our Skills

In [ ]:
display(Markdown("""
### 🔬 Applying Silver Gauge to Crisp Skills

Let's calculate k_explore for our three skills:
- peek_door: goal=0.0, info=1.0
- try_door: goal=1.0, info=0.0
- go_window: goal=1.0, info=0.0

What do you **predict** k_explore will be?
"""))

def calculate_silver_for_crisp():
    """Calculate Silver Gauge metrics for crisp skills"""
    skills_data = [
        {'name': 'peek_door', 'goal': 0.0, 'info': 1.0, 'cost': 1.0},
        {'name': 'try_door', 'goal': 1.0, 'info': 0.0, 'cost': 1.5},
        {'name': 'go_window', 'goal': 1.0, 'info': 0.0, 'cost': 2.0}
    ]
    
    results = []
    for skill in skills_data:
        k_exp = silver_k_explore(skill['goal'], skill['info'])
        benefit = skill['goal'] + skill['info']
        k_eff = silver_k_efficiency(benefit, skill['cost'])
        
        results.append({
            'Skill': skill['name'],
            'Goal': skill['goal'],
            'Info': skill['info'],
            'Cost': skill['cost'],
            'k_explore': k_exp,
            'k_efficiency': k_eff
        })
    
    return pd.DataFrame(results)

df_silver_crisp = calculate_silver_for_crisp()

print("\n" + "="*70)
print("SILVER GAUGE ANALYSIS: CRISP SKILLS")
print("="*70 + "\n")
print(df_silver_crisp.to_string(index=False))
print("\n" + "="*70)

display(Markdown("""
### 😱 WAIT... WHAT?!

Look at the **k_explore** column:

- **peek_door**: k ≈ 0.0001 (specialist!)
- **try_door**: k ≈ 0.0000 (specialist!)
- **go_window**: k ≈ 0.0000 (specialist!)

## 🎯 THE BIG DISCOVERY 🎯

**ALL crisp skills have k ≈ 0!**

### Why?

- **peek_door**: goal=0, info=1 → IMBALANCED → k≈0
- **try_door**: goal=1, info=0 → IMBALANCED → k≈0

**Both exploration AND exploitation are specialists!**

They're just specialists in **opposite directions**:
- peek = 100% info, 0% goal
- try = 100% goal, 0% info

### 💡 The Profound Insight

**k_explore doesn't measure "explore vs exploit"**

**k_explore measures "specialist vs generalist"**

- k≈0: Specialist (extreme in one direction)
- k≈1: Generalist (balanced multi-objective)

### ⚠️ The Gap

**No skills exist in the multi-objective zone (k > 0.5)!**

All our skills cluster at k≈0. There's a huge geometric gap!

This gap will inspire the next innovation...
"""))

## 4.7 Visualizing the Geometric Gap

In [ ]:
display(Markdown("""
### 📊 The Empty Multi-Objective Zone
"""))

# Scatter plot of skills in geometric space
fig, ax = plt.subplots(figsize=(14, 10))

for idx, row in df_silver_crisp.iterrows():
    k_exp = row['k_explore']
    k_eff = row['k_efficiency']
    skill_name = row['Skill']
    
    # Determine color by type
    if 'peek' in skill_name:
        color = COLORS['sense']
        marker = 'o'
    else:
        color = COLORS['act']
        marker = 's'
    
    # Plot point
    ax.scatter(k_exp, k_eff, s=600, alpha=0.8, color=color, marker=marker,
              edgecolors='black', linewidth=3, zorder=5)
    
    # Add label
    ax.annotate(skill_name, (k_exp, k_eff), fontsize=12, fontweight='bold',
               xytext=(15, 15), textcoords='offset points',
               bbox=dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.8),
               arrowprops=dict(arrowstyle='->', lw=2))

# Highlight zones
ax.axvspan(0.5, 1.0, alpha=0.15, color='green', label='Multi-objective zone (EMPTY!)', zorder=1)
ax.axvspan(0.0, 0.1, alpha=0.15, color='red', label='Specialist zone (ALL skills here!)', zorder=1)

# Add text annotations for zones
ax.text(0.75, 0.9, 'MULTI-OBJECTIVE\nZONE\n\n(EMPTY!)', 
       ha='center', va='center', fontsize=16, fontweight='bold',
       bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5, edgecolor='darkgreen', linewidth=3))

ax.text(0.05, 0.5, 'SPECIALIST\nZONE\n\n(crowded!)', 
       ha='center', va='center', fontsize=14, fontweight='bold',
       bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.5, edgecolor='darkred', linewidth=3))

ax.set_xlabel('k_explore (specialist ← 0 ... 1 → balanced)', fontsize=14, fontweight='bold')
ax.set_ylabel('k_efficiency (poor ← 0 ... 1 → excellent)', fontsize=14, fontweight='bold')
ax.set_title('Geometric Fingerprints of Crisp Skills\n⚠ GAP REVEALED: Multi-Objective Zone is Empty!',
            fontsize=16, fontweight='bold', pad=20)
ax.set_xlim(-0.05, 1.05)
ax.set_ylim(-0.05, 1.05)
ax.grid(alpha=0.3, linestyle='--')
ax.legend(fontsize=12, loc='upper right', framealpha=0.9)

# Add diagonal reference line
ax.plot([0, 1], [0, 1], 'k:', alpha=0.3, linewidth=1, label='k_explore = k_efficiency')

plt.tight_layout()
plt.show()

display(Markdown("""
### 🎯 What This Diagram Shows

**X-axis (k_explore)**:
- Left (k≈0): Specialist skills
- Right (k≈1): Balanced multi-objective skills

**Y-axis (k_efficiency)**:
- Bottom (k≈0): Poor benefit/cost ratio
- Top (k≈1): Excellent benefit/cost ratio

**The Gap**:
- ALL crisp skills cluster at x≈0 (specialist zone)
- NONE exist in x>0.5 (multi-objective zone)
- This is an **architectural gap**, not a bug!

---

## 🔬 The Diagnostic-Driven Design Pattern

**What just happened**:

1. ✅ Built sophisticated diagnostic (Silver Gauge)
2. ✅ Applied to system (crisp skills)
3. ✅ **Diagnostic revealed gap** (k ≈ 0 everywhere) ← WE ARE HERE
4. → Gap inspires solution (balanced skills) ← NEXT PART!
5. → Solution showcases diagnostic's value

This is a **general pattern** applicable beyond this project!
"""))

## 4.8 Checkpoint 4: Design a Balanced Skill

In [ ]:
display(Markdown("""
### ✅ Checkpoint 4: Your Turn!

**Challenge**: Design a skill with k_explore > 0.7

Adjust the sliders to create a balanced multi-objective skill:
"""))

goal_frac_slider = widgets.FloatSlider(
    value=0.5, min=0, max=1, step=0.05,
    description='Goal fraction:',
    style={'description_width': 'initial'}
)
info_frac_slider = widgets.FloatSlider(
    value=0.5, min=0, max=1, step=0.05,
    description='Info fraction:',
    style={'description_width': 'initial'}
)

challenge_output = widgets.Output()

def calculate_k_design(change):
    goal_frac = goal_frac_slider.value
    info_frac = info_frac_slider.value
    
    with challenge_output:
        clear_output(wait=True)
        
        k = silver_k_explore(goal_frac, info_frac)
        
        # Visual feedback
        fig, ax = plt.subplots(figsize=(12, 4))
        
        # Bar showing k value
        color = '#2ecc71' if k > 0.7 else ('#f39c12' if k > 0.5 else '#e74c3c')
        ax.barh(['k_explore'], [k], color=color, alpha=0.7, 
               edgecolor='black', linewidth=2, height=0.5)
        ax.set_xlim(0, 1)
        ax.set_xlabel('k_explore', fontsize=12, fontweight='bold')
        ax.axvline(x=0.7, color='green', linestyle='--', linewidth=2, 
                  label='Target (k=0.7)', alpha=0.7)
        ax.axvline(x=0.5, color='orange', linestyle='--', linewidth=2, 
                  label='Multi-objective threshold (k=0.5)', alpha=0.7)
        ax.legend(fontsize=10)
        ax.set_title(f'Your Skill Design: k_explore = {k:.4f}', 
                    fontsize=14, fontweight='bold')
        
        # Add value label
        ax.text(k, 0, f'{k:.3f}', ha='center', va='center',
               fontsize=14, fontweight='bold', color='white',
               bbox=dict(boxstyle='circle', facecolor='black'))
        
        plt.tight_layout()
        plt.show()
        
        # Text feedback
        print("="*70)
        print(f"Skill Design:")
        print(f"  Goal fraction: {goal_frac:.2f}")
        print(f"  Info fraction: {info_frac:.2f}")
        print(f"  k_explore: {k:.4f}")
        print("="*70)
        
        if k > 0.9:
            print("\n✓✓✓ EXCELLENT! Nearly perfect balance! (a ≈ b)")
            print("    This skill provides BOTH goal AND info effectively!")
        elif k > 0.7:
            print("\n✓✓ GREAT! Multi-objective skill achieved!")
            print("    You've filled the geometric gap!")
        elif k > 0.5:
            print("\n✓ GOOD! You've crossed into multi-objective territory!")
            print("    Can you get even more balanced?")
        else:
            print("\n⚠ Still too specialized.")
            print("    Hint: Both goal AND info need to be substantial!")

goal_frac_slider.observe(calculate_k_design, names='value')
info_frac_slider.observe(calculate_k_design, names='value')

display(goal_frac_slider, info_frac_slider, challenge_output)
calculate_k_design(None)  # Initial

display(Markdown("""
### 💡 Hint

To maximize k_explore:
- Need **both** goal and info to be high
- Perfect balance: goal = info = 0.7
- Try: goal=0.7, info=0.7 → k ≈ 1.0!
"""))

---

### 🎯 Part 4 Summary: THE REVELATION

**What we discovered**:

1. ✅ **Pythagorean means** (HM, GM, AM) create interpretable diagnostics
2. ✅ **k = GM/AM** is dimensionless, scale-invariant, bounded [0,1]
3. ✅ **k_explore** measures specialist (k≈0) vs generalist (k≈1)
4. ✅ **ALL crisp skills have k≈0** - even exploration and exploitation!
5. ✅ **Both are specialists** - just in opposite directions
6. ✅ **Geometric gap revealed** - no skills in multi-objective zone (k>0.5)
7. ✅ **100% behavioral fidelity** - diagnostic doesn't change EFE
8. ✅ **Diagnostic-driven design** - gap inspires solution

### 🤯 The Mind-Blowing Insight

**k_explore ≠ "explore vs exploit"**

**k_explore = "specialist vs generalist"**

Pure exploration (peek) and pure exploitation (try) are **both specialists** (k≈0)!

### 🎪 What's Next?

We've identified a geometric gap. How do we fill it?

→ **Multi-Objective Skills** (Balanced Skills)

Skills that provide **BOTH** goal achievement **AND** information gain!

---

**Next**: How do we create balanced skills? → **Multi-Objective Evolution!**

---

# PART 5: Multi-Objective Evolution - Balanced Skills

**Time**: 20 minutes

**Goals**:
- Understand compositional skill design
- Meet the four balanced skills
- Validate k_explore ∈ [0.56, 0.92]
- Visualize the complete geometric spectrum
- See how the gap is filled

**From gap to solution!**

## 5.1 The Diagnostic-Driven Design Pattern

In [ ]:
display(Markdown("""
### 🔄 From Gap to Solution

**The Pattern We Just Experienced**:

1. ✅ Build sophisticated diagnostic (Silver Gauge with Pythagorean means)
2. ✅ Apply diagnostic to existing system (crisp skills)
3. ✅ **Diagnostic reveals unexpected gap** (k ≈ 0 everywhere)
4. → **Gap inspires architectural innovation** (balanced skills) ← NOW!
5. → **Solution showcases diagnostic's value** (fills geometric spectrum)

### 💡 This is a GENERAL Meta-Pattern!

**Applicable beyond active inference**:
- Machine learning model interpretability
- Software architecture analysis
- System performance optimization
- Any domain where diagnostics reveal structure

---

### 🎯 The Question

We've discovered all crisp skills are specialists (k≈0).

**What if** skills could be multi-objective?

**What if** we could provide **BOTH** goal achievement **AND** information gain?

**How?** Compositional skill design!
"""))

## 5.2 Compositional Skill Design

In [ ]:
display(Markdown("""
### 🧩 Multi-Objective Skills via Composition

**Idea**: Create new skills as **weighted combinations** of base skills

**Example - probe_and_try**:
```
probe_and_try = 0.4 × peek_door + 0.6 × try_door
```

**Properties inherited** (compositionally):
- Cost: 0.4 × cost(peek) + 0.6 × cost(try)
       = 0.4 × 1.0 + 0.6 × 1.5 = 1.3
- Goal: 0.6 × goal(try) = 0.6 × 1.0 = 0.6
- Info: 0.4 × info(peek) = 0.4 × 1.0 = 0.4

**Predicted k_explore**:
```
k = GM(0.6, 0.4) / AM(0.6, 0.4)
  = sqrt(0.24) / 0.5
  = 0.4899 / 0.5
  ≈ 0.98  ← Nearly perfect balance!
```

### ✨ Benefits

1. **Fills geometric gap** (k > 0.5)
2. **Smooth spectrum** (continuous trade-offs)
3. **Compositional** (combines existing primitives)
4. **Interpretable** (fractions explicit)

Let's meet the four balanced skills!
"""))

## 5.3 The Four Balanced Skills

In [ ]:
display(Markdown("""
### 🌈 Introducing: Balanced Skills

Four new multi-objective skills:
"""))

# Define balanced skills
balanced_skills = [
    {'name': 'probe_and_try', 'cost': 2.0, 'goal': 0.6, 'info': 0.4},
    {'name': 'informed_window', 'cost': 2.3, 'goal': 0.8, 'info': 0.3},
    {'name': 'exploratory_action', 'cost': 2.5, 'goal': 0.7, 'info': 0.7},
    {'name': 'adaptive_peek', 'cost': 1.3, 'goal': 0.4, 'info': 0.6}
]

# Calculate k_explore for each
balanced_data = []
for skill in balanced_skills:
    k_exp = silver_k_explore(skill['goal'], skill['info'])
    balanced_data.append({
        'Skill': skill['name'],
        'Cost': skill['cost'],
        'Goal Fraction': skill['goal'],
        'Info Fraction': skill['info'],
        'k_explore': f"{k_exp:.4f}"
    })

df_balanced = pd.DataFrame(balanced_data)

display(df_balanced.style
        .background_gradient(subset=['Cost'], cmap='Reds')
        .background_gradient(subset=['Goal Fraction'], cmap='Greens')
        .background_gradient(subset=['Info Fraction'], cmap='Blues'))

display(Markdown("""
### 📊 Analysis

**1. probe_and_try** (k ≈ 0.98)
- 60% goal + 40% info
- Balanced approach: try first, learn from result
- Cost: 2.0 (medium)

**2. informed_window** (k ≈ 0.85)
- 80% goal + 30% info
- Goal-focused with information awareness
- Cost: 2.3 (higher)

**3. exploratory_action** (k = 1.00!)
- 70% goal + 70% info
- **Perfect balance!**
- Cost: 2.5 (highest)

**4. adaptive_peek** (k ≈ 0.92)
- 40% goal + 60% info
- Info-focused with goal awareness
- Cost: 1.3 (lowest)

**Key Result**: **ALL have k ∈ [0.85, 1.00]**

This fills the geometric gap!
"""))

## 5.4 The Complete Geometric Spectrum

In [ ]:
display(Markdown("""
### 🎨 Crisp + Balanced = Complete Toolkit

Let's visualize ALL skills together:
"""))

# Combine crisp and balanced
all_skills_data = []

# Add crisp skills
for _, row in df_silver_crisp.iterrows():
    k_exp = row['k_explore']
    k_eff = row['k_efficiency']
    all_skills_data.append({
        'name': row['Skill'],
        'kind': 'crisp',
        'k_explore': k_exp,
        'k_efficiency': k_eff
    })

# Add balanced skills
for skill in balanced_skills:
    k_exp = silver_k_explore(skill['goal'], skill['info'])
    benefit = skill['goal'] + skill['info']
    k_eff = silver_k_efficiency(benefit, skill['cost'])
    all_skills_data.append({
        'name': skill['name'],
        'kind': 'balanced',
        'k_explore': k_exp,
        'k_efficiency': k_eff
    })

# Plot
fig, ax = plt.subplots(figsize=(16, 10))

for skill_data in all_skills_data:
    if skill_data['kind'] == 'crisp':
        if 'peek' in skill_data['name']:
            color = COLORS['sense']
            marker = 'o'
            label = 'Crisp (sense)' if 'Crisp (sense)' not in [t.get_label() for t in ax.get_children()] else ''
        else:
            color = COLORS['act']
            marker = 's'
            label = 'Crisp (act)' if 'Crisp (act)' not in [t.get_label() for t in ax.get_children()] else ''
    else:
        color = COLORS['balanced']
        marker = '^'
        label = 'Balanced' if 'Balanced' not in [t.get_label() for t in ax.get_children()] else ''
    
    ax.scatter(skill_data['k_explore'], skill_data['k_efficiency'],
              s=700 if skill_data['kind'] == 'balanced' else 500,
              alpha=0.8, color=color, marker=marker,
              edgecolors='black', linewidth=3, zorder=5, label=label)
    
    # Annotate
    ax.annotate(skill_data['name'],
               (skill_data['k_explore'], skill_data['k_efficiency']),
               fontsize=11, fontweight='bold',
               xytext=(12, 12), textcoords='offset points',
               bbox=dict(boxstyle='round,pad=0.4', 
                        facecolor='lightgreen' if skill_data['kind'] == 'balanced' else 'white',
                        alpha=0.9, edgecolor='black', linewidth=1.5),
               arrowprops=dict(arrowstyle='->', lw=1.5))

# Highlight zones
ax.axvspan(0.0, 0.1, alpha=0.1, color='red', label='Specialist zone', zorder=1)
ax.axvspan(0.5, 1.0, alpha=0.1, color='green', label='Multi-objective zone', zorder=1)

# Add zone labels
ax.text(0.85, 0.15, '✓ GAP FILLED!\n\nBalanced skills\nhere now!', 
       ha='center', va='center', fontsize=14, fontweight='bold',
       bbox=dict(boxstyle='round', facecolor='lightgreen', 
                alpha=0.7, edgecolor='darkgreen', linewidth=3))

ax.set_xlabel('k_explore (specialist ← 0 ... 1 → balanced)', 
             fontsize=14, fontweight='bold')
ax.set_ylabel('k_efficiency (poor ← 0 ... 1 → excellent)', 
             fontsize=14, fontweight='bold')
ax.set_title('Complete Geometric Spectrum: Crisp + Balanced Skills\n✨ Full Coverage Achieved!',
            fontsize=17, fontweight='bold', pad=20)
ax.set_xlim(-0.05, 1.05)
ax.set_ylim(-0.05, 1.05)
ax.grid(alpha=0.3, linestyle='--')
ax.legend(fontsize=12, loc='upper left', framealpha=0.95)

plt.tight_layout()
plt.show()

display(Markdown("""
## 🎉 SUCCESS! 🎉

**Before**: All skills clustered at k ≈ 0 (specialist zone)

**After**: Full geometric spectrum from k=0 to k≈1.0

### 💡 The Innovation

**Complementarity over Replacement**:
- **Crisp skills**: Pedagogical clarity (sharp boundaries)
- **Balanced skills**: Analytical richness (smooth spectrum)
- **Together**: Complete toolkit for active inference

We didn't replace crisp skills - we AUGMENTED them!

### 🔬 What This Enables

1. **Geometric curriculum learning**: Progress through k values
2. **Transfer learning**: k patterns work across domains
3. **Meta-learning**: Learn to adjust k based on task
4. **Multi-agent coordination**: Assign roles via k profiles
5. **Interpretable AI**: Understand strategies geometrically

---

### 🌟 The Complete Picture

**Crisp skills** (k≈0):
- peek_door: Pure info gathering
- try_door: Pure goal seeking
- go_window: Guaranteed exit

**Balanced skills** (k>0.8):
- adaptive_peek: Info-focused multi-objective
- probe_and_try: Balanced exploration-exploitation
- exploratory_action: Perfect balance!
- informed_window: Goal-focused multi-objective

**Result**: Complete k spectrum coverage!
"""))

---

# PART 6: Neo4j Deep Dive & Query Playground

**Time**: 15 minutes

**Goals**:
- Learn Cypher query language basics
- Explore the database interactively
- Run your own custom queries
- Visualize query results
- Understand the knowledge graph deeply

**Your turn to explore!**

## 6.1 Cypher Query Language Basics

In [ ]:
display(Markdown("""
### 📖 Cypher: Graph Query Language

Neo4j uses **Cypher** - an intuitive, ASCII-art style query language.

**Core Concepts**:

#### 1. Nodes (Entities)
```cypher
(n)              // Any node
(s:Skill)        // Node with label 'Skill'
(s:Skill {name: 'peek_door'})  // Node with properties
```

#### 2. Relationships (Connections)
```cypher
-[r]->           // Any relationship (directed)
-[:CAN_USE]->    // Relationship with type
-[r:CAN_USE {confidence: 0.9}]->  // With properties
```

#### 3. Patterns (Queries)
```cypher
(a)-[:KNOWS]->(b)  // Pattern: a knows b
(a)-[:KNOWS*1..3]->(b)  // Path: 1-3 hops
```

#### 4. Common Clauses
```cypher
MATCH (n:Label)      // Find nodes
WHERE n.prop > 5     // Filter
RETURN n             // Return results
ORDER BY n.prop DESC // Sort
LIMIT 10             // Limit results
```

---

### 📚 Example Queries for Our Database
"""))

if NEO4J_CONNECTED:
    # Show example queries
    examples = [
        {
            'description': '1. Get all skills',
            'query': 'MATCH (s:Skill) RETURN s.name, s.kind, s.cost ORDER BY s.cost',
            'explanation': 'Find all nodes labeled Skill, return their properties'
        },
        {
            'description': '2. Find crisp (non-balanced) skills',
            'query': "MATCH (s:Skill) WHERE s.kind <> 'balanced' RETURN s.name, s.kind",
            'explanation': 'Filter skills where kind is NOT balanced'
        },
        {
            'description': '3. Get procedural memories',
            'query': 'MATCH (m:Memory)-[r:RECOMMENDS]->(s:Skill) RETURN m.context, s.name, r.confidence LIMIT 5',
            'explanation': 'Find Memory-RECOMMENDS->Skill patterns'
        },
        {
            'description': '4. Count nodes by label',
            'query': 'MATCH (n) RETURN labels(n)[0] as label, count(*) as count ORDER BY count DESC',
            'explanation': 'Aggregate count of each node type'
        }
    ]
    
    for ex in examples:
        print("\n" + "="*70)
        print(ex['description'])
        print("="*70)
        print("\nQuery:")
        print(ex['query'])
        print("\nExplanation:")
        print(ex['explanation'])
        print("\nResults:")
        
        try:
            results = run_query(ex['query'])
            if results:
                df = pd.DataFrame(results)
                display(df)
            else:
                print("  (No results)")
        except Exception as e:
            print(f"  Error: {e}")
else:
    print("\n⚠ Neo4j not connected. Examples shown conceptually.")
    print("\nConnect to Neo4j to run live queries!")

## 6.2 🎮 Interactive Query Playground

In [ ]:
display(Markdown("""
### 💻 Try Your Own Cypher Queries!

**Suggested Queries to Try**:

1. **All skills**: `MATCH (s:Skill) RETURN s`
2. **High-cost skills**: `MATCH (s:Skill) WHERE s.cost > 1.5 RETURN s.name, s.cost`
3. **Balanced skills only**: `MATCH (s:Skill {kind: 'balanced'}) RETURN s.name`
4. **State transitions**: `MATCH (s1:State)-[r]->(s2:State) RETURN s1.name, type(r), s2.name`
5. **Skill network**: `MATCH path = (s1:Skill)-[*1..2]-(s2:Skill) RETURN path LIMIT 5`

**Tips**:
- Start with simple `MATCH ... RETURN` queries
- Use `LIMIT` to avoid huge results
- Check syntax if you get errors
- Neo4j is case-sensitive for labels!
"""))

# Interactive query widget
query_text = widgets.Textarea(
    value='MATCH (s:Skill) RETURN s.name, s.kind, s.cost ORDER BY s.cost',
    placeholder='Enter Cypher query here...',
    description='Cypher Query:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='90%', height='100px')
)

run_query_button = widgets.Button(
    description='Run Query',
    button_style='success',
    icon='play'
)

clear_button = widgets.Button(
    description='Clear',
    button_style='warning'
)

visualize_checkbox = widgets.Checkbox(
    value=False,
    description='Visualize as graph (if applicable)',
    style={'description_width': 'initial'}
)

query_output = widgets.Output()

def on_run_query(b):
    with query_output:
        clear_output(wait=True)
        
        query = query_text.value.strip()
        
        if not query:
            print("⚠ Please enter a query!")
            return
        
        if not NEO4J_CONNECTED:
            print("❌ Neo4j not connected!")
            print("   Start Neo4j: make neo4j-start")
            print("   Initialize DB: make init")
            return
        
        print("🔍 Running query...")
        print("=" * 70)
        print(query)
        print("=" * 70)
        print()
        
        try:
            results = run_query(query)
            
            if not results:
                print("✓ Query succeeded but returned no results.")
                return
            
            print(f"✓ Query returned {len(results)} result(s)\n")
            
            # Show as DataFrame
            df = pd.DataFrame(results)
            print("Results as Table:")
            display(df)
            
            # Optionally visualize
            if visualize_checkbox.value:
                print("\nGraph Visualization:")
                
                # Try to build graph from results
                G_result = nx.DiGraph()
                
                # Look for node/relationship patterns
                for row in results:
                    # Try to find source/target pairs
                    keys = list(row.keys())
                    if len(keys) >= 2:
                        src = str(row[keys[0]])
                        dst = str(row[keys[1]])
                        label = keys[2] if len(keys) > 2 else ''
                        G_result.add_edge(src, dst, label=str(label))
                
                if G_result.number_of_nodes() > 0:
                    plt.figure(figsize=(12, 8))
                    pos = nx.spring_layout(G_result, k=1, iterations=50)
                    nx.draw(G_result, pos, with_labels=True, 
                           node_color='lightblue', node_size=2000,
                           font_size=10, font_weight='bold',
                           arrows=True, edge_color='gray', width=2)
                    
                    if G_result.number_of_edges() < 20:  # Only show labels if not too many
                        edge_labels = nx.get_edge_attributes(G_result, 'label')
                        nx.draw_networkx_edge_labels(G_result, pos, edge_labels, font_size=8)
                    
                    plt.title('Query Result Graph', fontsize=14, fontweight='bold')
                    plt.axis('off')
                    plt.tight_layout()
                    plt.show()
                else:
                    print("  (Results not suitable for graph visualization)")
        
        except Exception as e:
            print(f"❌ Query Error: {e}")
            print("\nCommon issues:")
            print("  - Check syntax (case-sensitive!)")
            print("  - Label or property doesn't exist")
            print("  - Missing RETURN clause")

def on_clear(b):
    with query_output:
        clear_output()
    query_text.value = ''

run_query_button.on_click(on_run_query)
clear_button.on_click(on_clear)

display(widgets.VBox([
    query_text,
    widgets.HBox([run_query_button, clear_button]),
    visualize_checkbox
]))
display(query_output)

display(Markdown("""
### 💡 Pro Tips

**Explore the schema**:
```cypher
CALL db.labels()  // All node labels
CALL db.relationshipTypes()  // All relationship types
```

**Find example of each type**:
```cypher
MATCH (n:Skill) RETURN n LIMIT 1
```

**Explore relationships**:
```cypher
MATCH (n)-[r]->(m) RETURN type(r) as rel_type, count(*) as count GROUP BY rel_type
```
"""))

## 6.3 Common Query Patterns

In [ ]:
display(Markdown("""
### 📚 Query Pattern Library

Here are some useful patterns for exploring the MacGyver MUD database:
"""))

patterns = [
    {
        'name': 'Filter by Property',
        'query': 'MATCH (s:Skill) WHERE s.cost < 2.0 RETURN s.name, s.cost',
        'use': 'Find low-cost skills'
    },
    {
        'name': 'Aggregate Functions',
        'query': 'MATCH (s:Skill) RETURN AVG(s.cost) as avg_cost, MAX(s.cost) as max_cost',
        'use': 'Statistical summaries'
    },
    {
        'name': 'Relationship Patterns',
        'query': 'MATCH (a:Agent)-[:CAN_USE]->(s:Skill) RETURN a.name, s.name',
        'use': 'Which skills can the agent use?'
    },
    {
        'name': 'Path Finding',
        'query': "MATCH path = (start:State {name:'stuck_in_room'})-[:LEADS_TO*]->(end:State) WHERE 'escaped' IN end.name RETURN path LIMIT 3",
        'use': 'Find escape paths'
    },
    {
        'name': 'Count Relationships',
        'query': 'MATCH ()-[r:RECOMMENDS]->() RETURN count(r) as total_memories',
        'use': 'How many procedural memories exist?'
    },
    {
        'name': 'Property Exists',
        'query': 'MATCH (s:Skill) WHERE exists(s.goal_fraction) RETURN s.name',
        'use': 'Find balanced skills (have goal_fraction property)'
    }
]

print("\n📖 Query Pattern Library\n")
for i, pattern in enumerate(patterns, 1):
    print("=" * 70)
    print(f"{i}. {pattern['name']}")
    print("=" * 70)
    print(f"Use case: {pattern['use']}")
    print(f"\nQuery:\n{pattern['query']}")
    print()

display(Markdown("""
### 🎯 Try These Challenges

Can you write queries to:

1. Find the most expensive skill?
2. Count how many balanced vs crisp skills exist?
3. Find all observations that skills can produce?
4. Get the agent's name and current state?
5. Find which memories recommend 'peek_door'?

Use the playground above to experiment!
"""))

## 6.4 Advanced: Visualizing Complex Queries

In [ ]:
display(Markdown("""
### 🎨 Create Custom Visualizations

Example: Visualize the entire skill network with properties
"""))

if NEO4J_CONNECTED:
    # Complex visualization example
    skills_detailed = run_query("""
        MATCH (s:Skill)
        OPTIONAL MATCH (s)-[r]->(o:Observation)
        RETURN s.name as skill, 
               s.kind as kind, 
               s.cost as cost,
               collect(o.name) as observations
    """)
    
    if skills_detailed:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        # Left: Skills by cost and kind
        df_skills = pd.DataFrame(skills_detailed)
        
        crisp_skills = df_skills[df_skills['kind'] != 'balanced']
        balanced_skills = df_skills[df_skills['kind'] == 'balanced']
        
        if not crisp_skills.empty:
            ax1.barh(crisp_skills['skill'], crisp_skills['cost'], 
                    color='#3498db', alpha=0.7, label='Crisp')
        if not balanced_skills.empty:
            ax1.barh(balanced_skills['skill'], balanced_skills['cost'],
                    color='#2ecc71', alpha=0.7, label='Balanced')
        
        ax1.set_xlabel('Cost', fontsize=12)
        ax1.set_title('Skills by Cost', fontsize=14, fontweight='bold')
        ax1.legend()
        ax1.grid(axis='x', alpha=0.3)
        
        # Right: Skill counts by kind
        kind_counts = df_skills['kind'].value_counts()
        ax2.pie(kind_counts.values, labels=kind_counts.index, autopct='%1.1f%%',
               colors=['#3498db', '#2ecc71', '#e74c3c'])
        ax2.set_title('Skill Distribution by Kind', fontsize=14, fontweight='bold')
        
        plt.tight_layout()
        plt.show()
        
        print("\n📊 Skills Overview:")
        print(f"  Total skills: {len(df_skills)}")
        print(f"  Crisp skills: {len(crisp_skills)}")
        print(f"  Balanced skills: {len(balanced_skills)}")
        print(f"  Average cost: {df_skills['cost'].mean():.2f}")
    else:
        print("⚠ No skills found in database.")
else:
    print("⚠ Neo4j not connected.")

display(Markdown("""
### 🚀 What You Can Do

With Neo4j and Cypher, you can:

1. **Explore**: Query any part of the knowledge graph
2. **Analyze**: Aggregate statistics and patterns
3. **Visualize**: Create custom graphs and charts
4. **Learn**: Understand how data is structured
5. **Experiment**: Test new queries and relationships

**Neo4j is your window into the agent's world!**
"""))

---

### 🎯 Part 6 Summary

**What we learned**:

1. ✅ **Cypher basics** - Query language syntax and patterns
2. ✅ **Interactive playground** - Run your own queries!
3. ✅ **Common patterns** - Useful query templates
4. ✅ **Visualization** - Turn queries into graphs and charts
5. ✅ **Database exploration** - Full access to knowledge graph

### 💡 Key Takeaways

- **Cypher is intuitive**: ASCII-art style patterns
- **Graph structure matters**: Relationships enable powerful queries
- **Exploration is easy**: Try queries without breaking anything
- **Visualization helps**: See patterns in the data

### 🔗 Connection to Active Inference

The knowledge graph stores:
- **Skills** → What the agent can do
- **States** → Where the agent can be
- **Observations** → What the agent perceives
- **Memories** → What the agent has learned

All of this enables the agent to:
1. Plan (find paths)
2. Reason (query relationships)
3. Learn (store experiences)
4. Adapt (update beliefs)

**The graph IS the agent's knowledge!**

---

**Next**: Back to our journey... we've seen execution and explored the database. Now let's understand strategies geometrically with the Silver Gauge!

---

# 🎓 FINAL SUMMARY: What We've Learned

## The Journey

### Part 1: The Problem
- Locked room scenario
- Uncertainty matters
- Skills have trade-offs
- Beliefs affect decisions

### Part 2: The Math
- Expected Free Energy (EFE) = cost - goal - info
- Beliefs are probability distributions
- Lower EFE = better choice
- Active inference balances exploration/exploitation naturally

### Part 3: The Application
- Score all three skills
- Decision boundaries emerge
- Crossover points show policy switches

### Part 4: THE REVELATION ⭐
- Pythagorean means (HM, GM, AM)
- k = GM/AM coefficients
- **ALL crisp skills have k≈0** (specialists!)
- BOTH exploration AND exploitation are specialists
- Geometric gap revealed
- Diagnostic-driven design pattern

### Part 5: The Solution
- Compositional skill design
- Four balanced skills
- k ∈ [0.85, 1.00] for balanced skills
- Gap filled!
- Complete geometric spectrum

---

## Key Insights

### 1. The k≈0 Phenomenon
**Discovery**: ALL crisp skills are specialists (k≈0), whether exploration OR exploitation.

**Why**: Both are imbalanced:
- peek: 100% info, 0% goal → k≈0
- try: 100% goal, 0% info → k≈0

**Implication**: k_explore measures specialist vs generalist, NOT explore vs exploit!

### 2. Diagnostic-Driven Design
**Pattern**:
1. Build diagnostic (Silver Gauge)
2. Apply to system (crisp skills)
3. Diagnostic reveals gap (k≈0 everywhere)
4. Gap inspires solution (balanced skills)
5. Solution showcases diagnostic

**This generalizes beyond active inference!**

### 3. Complementarity Over Replacement
**Insight**: Don't replace crisp with balanced - maintain both!

**Rationale**:
- Crisp: Pedagogical clarity
- Balanced: Analytical richness
- Together: Complete toolkit

### 4. Scale-Invariant Transfer
**Insight**: Dimensionless k enables cross-domain transfer

**Example**:
- Learn: k_explore > 0.6 early → success
- Apply to new domain → same pattern works!

---

## Research Directions

1. **Geometric Curriculum Learning**: Progress through k values
2. **Transfer Learning**: Build reusable strategy libraries
3. **Meta-Learning**: Learn to adjust k based on task
4. **Multi-Agent**: Assign roles via geometric profiles
5. **Anomaly Detection**: Detect when k patterns deviate
6. **Continuous Skills**: Infinite resolution in k-space
7. **Hierarchical Geometries**: Multi-scale k coefficients
8. **Deep RL Integration**: Use k as auxiliary objectives

---

## Mathematical Beauty

**Ancient math (500 BCE) applied to modern AI (2025)**:
- Pythagorean means
- Dimensionless ratios
- Scale-invariant metrics
- 100% behavioral fidelity
- Interpretable without sacrifice

**Innovation Level**: 7/10
- Novel application of classical mathematics
- Solves interpretability without approximation
- Reveals hidden architectural structure

---

## Further Exploration

### Documentation
- `FINAL_REPORT.md` - 75-page comprehensive analysis
- `PYTHAGOREAN_MEANS_EXPLAINED.md` - Mathematical deep dive
- `BALANCED_POLICY_GUIDE.md` - Multi-objective guide
- `README.md` - Project overview

### Code
- `graph_model.py` - Neo4j operations
- `scoring_silver.py` - Silver Gauge implementation
- `scoring_balanced.py` - Balanced skills
- `runner.py` - Run episodes with --skill-mode

### Try It Yourself
```bash
# Run with different skill modes
python runner.py --door-state locked --skill-mode crisp
python runner.py --door-state locked --skill-mode balanced
python runner.py --door-state locked --skill-mode hybrid

# Generate visualizations
make visualize-balanced
```

---

## Thank You!

You've completed the deep dive into:
- Active Inference
- Geometric Diagnostics
- Multi-Objective Evolution
- Diagnostic-Driven Design

**Key Takeaway**: Classical mathematics + modern AI = interpretable intelligence

---

*"Measure what is measurable, and make measurable what is not so." — Galileo*

*We've made decision strategies measurable through geometry.*

🎉 **Congratulations on finishing the notebook!** 🎉